# *__Working on BTCUSD predictions with GRU model.__*

## __Check first before starting__

In [ ]:
import os

# Change the working directory to the project root
Working_directory = os.path.normpath("C:/Users/gilda/OneDrive/Documents/_NYCU/MASTER_S_studies/Master\'s Thesis/LABORATORY/_Global_Pytorch/Continual_Learning")
os.chdir(Working_directory)
print(f"Working directory: {os.getcwd()}")

Working directory: C:\Users\gilda\OneDrive\Documents\_NYCU\MASTER_S_studies\Master's Thesis\LABORATORY\_Global_Pytorch\Continual_Learning


## **__All imports__**

In [5]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_interactions import zoom_factory, panhandler
from sklearn.model_selection import train_test_split
import pickle
from ta import trend, momentum, volatility, volume
import math
from scipy.ndimage import gaussian_filter1d
from typing import Callable, Tuple, Union
import shutil
import contextlib
import traceback
import gc
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

## __**All functions (For data processing)**__

In [ ]:
def ensure_folder(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

def plot_with_matplotlib(data: pd.DataFrame, 
                         title: str, 
                         interactive: bool = False, 
                         save_path: str = None, 
                         show_plot: bool = True, 
                         save_matplotlib_object: str = None) -> None:
    """
    Plot data using Matplotlib, with optional interactivity using mpl-interactions.
    
    Parameters:
    - data (pd.DataFrame): The data to plot, must contain 'close' column.
    - title (str): The title of the plot.
    - interactive (bool): If True, enables interactive zoom and pan.
    - save_path (Optional[str]): If provided, saves the plot to this path.
    - show_plot (bool): If True, displays the plot. If False, skips display.
    - save_matplotlib_object (Optional[str]): If provided, saves the Matplotlib figure object to this file path.
    """
    if not all(col in data.columns for col in ['close']):
        raise ValueError("The input DataFrame must contain 'close' column.")

    # Use the default Matplotlib color cycle for the line
    default_blue = plt.rcParams['axes.prop_cycle'].by_key()['color'][0]
    print(f"Default blue: {default_blue}")

    # Explicit colors for trends
    trend_colors = {
        0: 'black',
        1: 'yellow',
        2: 'red',
        3: 'green',
        4: default_blue #'purple',
    }
    # unique_trends = [0, -25, -15, 15, 25]
    # colormap = plt.cm.get_cmap('tab10', len(unique_trends))  # Choose 'tab10' or 'Set1' for distinct colors
    # trend_colors = {trend: colormap(i) for i, trend in enumerate(unique_trends)}

    fig, ax = plt.subplots(figsize=(12, 6))

    # Plot data as a single connected line, colored by trend
    if 'trend' in data.columns:
        legend_added = set() # Track which trends have already been added to the legend
        prev_idx = data.index[0]
        for idx, row in data.iterrows():
            if idx != prev_idx:
                trend_key = int(row['trend'])  # Convert trend value to int for lookup
                label = f'Trend {trend_key}' if trend_key not in legend_added else None
                ax.plot([prev_idx, idx], 
                        [data.loc[prev_idx, 'close'], row['close']],
                        color=trend_colors[trend_key], 
                        linestyle='-', 
                        # marker='o', 
                        linewidth=1,
                        label=label  # Add label only if it's not in the legend
                )
                legend_added.add(trend_key)  # Mark this trend as added to the legend
            prev_idx = idx

        ax.set_title(f"{title} (Connected, Colored by Trend)")
    else:
        # Default plot if no trend column exists
        ax.plot(data.index, data['close'], label='Closing Price', linestyle='-', marker='o', 
                markersize=2, linewidth=1, color=default_blue, markerfacecolor='green', markeredgecolor='black')
        ax.set_title(title)
    
    ax.set_xlabel('Date')
    ax.set_ylabel('Closing Price (USD)')
    
    # Add a legend manually for trends
    # for trend, color in trend_colors.items():
    #     ax.plot([], [], color=color, label=f'Trend {trend}')
    ax.legend()
    ax.grid()
    
    # Enable interactivity if requested
    if interactive:
        zoom_factory(ax)  # Enable zoom with mouse wheel
        panhandler(fig)   # Enable panning with left-click
        print("Interactive mode enabled. Use mouse wheel to zoom and left click to pan.")

    # Save the plot if a path is provided
    if save_path:
        fig.tight_layout()  # Ensures the layout is clean
        fig.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Plot saved to: {save_path}")

    # Save the Matplotlib figure object
    if save_matplotlib_object:
        with open(save_matplotlib_object, 'wb') as f:
            pickle.dump(fig, f)
        print(f"Matplotlib figure object saved to: {save_matplotlib_object}")

    if show_plot:
        plt.show()
    else:
        print("Plot display skipped.")

def load_and_show_pickle(pickle_file_path: str):
    """
    Load a pickled Matplotlib figure object and display it with optional interactivity.

    Parameters:
    - pickle_file_path (str): Path to the pickled Matplotlib figure file.

    Returns:
    - None
    """
    try:
        # Open the pickle file and load the figure
        with open(pickle_file_path, "rb") as f:
            loaded_fig = pickle.load(f)

        print(f"Figure successfully loaded and displayed from: {pickle_file_path}")

        # Use plt.show() to allow interactivity
        plt.show(block=True)

    except FileNotFoundError:
        print(f"Error: File not found at {pickle_file_path}. Please check the path.")
    except Exception as e:
        print(f"Error loading the pickled figure: {e}")

def save_to_csv(df: pd.DataFrame, file_path: str):
    """
    Save a DataFrame to a CSV file.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the data to be saved.
        file_path (str): The file path (including the file name) to save the CSV.

    Returns:
        None
    """
    df.to_csv(file_path)
    # df_to_save = df.copy()
    # df_to_save["date"] = df_to_save.index.strftime('%Y-%m-%d %H:%M:%S')  # Add formatted index as a column
    
    # Save the DataFrame to CSV
    # df_to_save.to_csv(file_path)
    print(f"\nSuccessfully saved data with moving average to CSV: \n\t{file_path}\n")

def read_csv_file(file_path: str, preview_rows: int = 5, 
                  days_towards_end: int = None, 
                  days_from_start: int = None, description: str = ""):
    """
    Reads a CSV file and returns a pandas DataFrame filtered by date range.

    Args:
        file_path (str): The path to the CSV file.
        preview_rows (int): Number of rows to preview (default is 5).
        days_towards_end (int, optional): Number of days from the most recent date to retrieve data.
        days_from_start (int, optional): Number of days from the oldest date of the filtered data to retrieve data.
        description (str): A brief description of the dataset being loaded.
                           Explanation:
                           - To retrieve data from the **end**: Use `days_towards_end`.
                           - To retrieve data from the **start of the filtered range**: Use `days_from_start`.
                           - To retrieve data from the **middle**: Use both:
                             For example, if `days_towards_end=100` and `days_from_start=50`,
                             the function will first filter the last 100 days of the dataset,
                             and then filter the first 50 days from this range.
                             This results in data between the last 100th and the last 50th day.

    Returns:
        DataFrame: The loaded and filtered data from the CSV file.
    """
    try:
        if description:
            print(f"\nDescription: {description}")
        print(f"\nFile path: {file_path}")
        
        # Read the CSV file
        data = pd.read_csv(file_path, parse_dates=['date'], index_col='date')
        
        # Filter by days towards the end
        if days_towards_end is not None:
            last_date = data.index.max()  # Get the most recent date in the dataset
            end_cutoff_date = last_date - pd.Timedelta(days=days_towards_end)
            data = data[data.index >= end_cutoff_date]
            print(f"\nRetrieving data from the past {days_towards_end} days (from {end_cutoff_date.date()} onwards):")
        
        # Filter by days from the start (from the filtered data)
        if days_from_start is not None:
            first_date = data.index.min()  # Get the earliest date in the filtered dataset
            start_cutoff_date = first_date + pd.Timedelta(days=days_from_start)
            data = data[data.index <= start_cutoff_date]
            print(f"\nRetrieving the first {days_from_start} days from the filtered data (up to {start_cutoff_date.date()}):")

        if preview_rows:
            # Print a preview of the data
            print(f"\nPreview of the first {preview_rows} rows:")
            # print(data.head(preview_rows), '\n')
            display(data.head(preview_rows))
            print()

            print(f"\nPreview of the last {preview_rows} rows:")
            # print(data.tail(preview_rows), '\n')
            display(data.tail(preview_rows))
            print()

        return data
    except FileNotFoundError:
        print("Error: File not found. Please check the file path.")
    except pd.errors.EmptyDataError:
        print("Error: The file is empty.")
    except pd.errors.ParserError:
        print("Error: The file could not be parsed. Please check the file format.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def downsample_minute_data(data: pd.DataFrame, n: int) -> pd.DataFrame:
    """
    Downsample minute data into N-minute intervals by retaining every Nth row.

    Parameters:
        data (pd.DataFrame): The original DataFrame with a datetime index.
        n (int): The number of minutes for the downsampling interval.

    Returns:
        pd.DataFrame: Downsampled DataFrame.
    """
    print("\n========---> Downsampling the data! \n")
    data = data.copy()
    # Ensure the index is a DatetimeIndex
    if not isinstance(data.index, pd.DatetimeIndex):
        try:
            data.index = pd.to_datetime(data.index)  # Convert to DatetimeIndex
        except Exception as e:
            raise ValueError("The DataFrame index could not be converted to DatetimeIndex.") from e

    # Filter rows where the minute index modulo N is 0
    downsampled_data = data[data.index.minute % n == 0]

    return downsampled_data

def calculate_log_returns_all_columns(data: pd.DataFrame, exclude_columns: list = [], dropna: bool = True) -> pd.DataFrame:
    """
    Calculate log returns for all numeric columns in a pandas DataFrame,
    excluding specified columns, and removing excluded columns from the returned DataFrame.

    Args:
        data (pd.DataFrame): Input DataFrame containing numeric data.
        exclude_columns (list): List of columns to exclude from log return calculations and the result.
        dropna (bool): Whether to drop rows with NaN values resulting from the calculation.

    Returns:
        pd.DataFrame: DataFrame with log returns for numeric columns,
                      excluding specified columns.
    """
    data = data.copy().drop(columns=exclude_columns)
    columns_to_transform = data.select_dtypes(include=[np.number]).columns
    print(f"columns_to_transform = \n{columns_to_transform}, \nlen(columns_to_transform) = {len(columns_to_transform)}")

    for col in columns_to_transform:
        # Ensure no negative or zero values
        if (data[col] <= 0).any():
            raise ValueError(f"Column '{col}' contains non-positive values. Log returns require strictly positive values.")
        data[col] = np.log(data[col] / data[col].shift(1))

    # Optionally drop rows with NaN values
    return data.dropna() if dropna else data

def created_sequences_2(data: pd.DataFrame, sequence_length: int = 60, sliding_interval: int = 60) -> list:
    """
    Divide the dataset into sequences based on the sequence_length.
    Each sequence must fully cover the window size.

    Args:
    - data (pd.DataFrame): The input DataFrame.
    - sequence_length (int): The window size for sequences.

    Returns:
    - sequences (list): A list of sequences (as DataFrames).
    """
    sequences = []
    for i in range(0, len(data) - sequence_length + 1, sliding_interval):
        # print(f"Processing sequence starting at index: {i}")
        seq = data.iloc[i:i + sequence_length].copy()
        sequences.append(seq)
    # print(f"Total sequences created: {len(sequences)}")
    return sequences

def gaussian_smoothing(data: pd.DataFrame, sigma=2) -> pd.DataFrame:
    """
    Applies Gaussian smoothing to numeric columns in a DataFrame and ensures the index is sorted in ascending order.

    Args:
        data (pd.DataFrame): Input DataFrame.
        sigma (float): Standard deviation for the Gaussian kernel. Defaults to 2.

    Returns:
        pd.DataFrame: A DataFrame with smoothed numeric columns and sorted index.
    """
    # Sort the DataFrame by index in ascending order
    data = data.sort_index(ascending=True).copy()
    for column in data.columns:
        if pd.api.types.is_numeric_dtype(data[column]):  # Only apply to numeric columns
            data[column] = gaussian_filter1d(data[column].values, sigma=sigma)
    return data

def detect_trends_4(
    dataframe: pd.DataFrame, 
    column: str = 'close', 
    lower_threshold: float = 0.001, 
    upper_threshold: float = 0.02,
    reverse_steps: int = 7,
    trends_to_keep: set = {0, 1, 2, 3, 4}  # Default keeps all trends
) -> pd.DataFrame:
    """
    Detects trends based on log return data provided in a specified column and categorizes them into different strength levels.

    This function analyzes time-series data by evaluating cumulative trends in log return values provided in the input DataFrame. 
    It uses three dictionaries (`dic1`, `dic2`, `dic3`) to track different phases of trends, handles multi-step reversals, and 
    classifies trends dynamically based on cumulative product thresholds and specified thresholds for trend strengths.

    Args:
        dataframe (pd.DataFrame): Input DataFrame containing log return data.
        column (str): Column name containing log return values. Defaults to 'close'.
        lower_threshold (float): Threshold for categorizing moderate trends. Defaults to 0.001.
        upper_threshold (float): Threshold for categorizing strong trends. Defaults to 0.02.
        reverse_steps (int): Number of consecutive steps to confirm a trend reversal. Defaults to 7.
        trends_to_keep (set): A set of trend categories to retain; others will be set to 0 (No Trend). Defaults to keeping all trends {0, 1, 2, 3, 4}.

    Returns:
        pd.DataFrame: A DataFrame with an added column:
                    - 'trend': Categorized trend values based on the detected phases:
                        - 0: No trend
                        - 1: Moderate negative trend
                        - 2: Very strong negative trend
                        - 3: Moderate positive trend
                        - 4: Very strong positive trend
                      Any trends not included in `trends_to_keep` will be reset to 0.

    Function Details:
    1. **Input Assumption**:
    - The input DataFrame already contains log return data in the specified column (`column`).

    2. **Trend Tracking**:
    - Uses dictionaries to monitor trends:
        - `dic1`: Tracks the first phase of the trend.
        - `dic2`: Tracks the second phase if a reversal occurs.
        - `dic3`: Tracks the third phase if another reversal occurs.

    3. **Cumulative Product**:
    - Calculates the cumulative product of `(1 + log_return)` from the specified column to evaluate the strength of trends.

    4. **Reversal Handling**:
    - If a trend reversal persists beyond `reverse_steps`, labels are assigned based on the cumulative product tracked in `dic1`.
    - Subsequent reversals are merged or labeled independently if conditions are met.

    5. **Label Assignment**:
    - Labels are dynamically assigned based on cumulative product thresholds for positive and negative trends:
        - Positive trends are categorized as moderate (3) or strong (4).
        - Negative trends are categorized as moderate (1) or strong (2).

    6. **Trend Filtering**:
    - After detecting trends, only those specified in `trends_to_keep` remain unchanged.
    - Any trend category not included in `trends_to_keep` is reset to 0 (No Trend).

    7. **Edge Cases**:
    - Properly handles scenarios where data points are insufficient for trend analysis or when trend phases overlap, ensuring all data points are labeled.
    """
    # Copy to avoid modifying the original DataFrame
    df = dataframe.copy()
    df['trend'] = None  # Default value 
    
    # print("\n#-------------------- Working on 'trend' patterns -----------------------#")
    dic1, dic2, dic3 = None, None, None # Initialize trend tracking dictionaries
    # dic1 = None # {'ids': [], 'last_sign': None, 'cumulative': 1.0}
    
    def assign_label(dictio_, lower_threshold, upper_threshold):
        cumulative = dictio_['cumulative']
        # print(f"cumulative = {cumulative}")
        if cumulative > (1 + upper_threshold):
            df.iloc[dictio_['ids'], df.columns.get_loc('trend')] = 4  # Very strong positive
        elif (1 + lower_threshold) < cumulative <= (1 + upper_threshold):
            df.iloc[dictio_['ids'], df.columns.get_loc('trend')] = 3  # Moderate positive
        elif (1 - upper_threshold) < cumulative <= (1 - lower_threshold):
            df.iloc[dictio_['ids'], df.columns.get_loc('trend')] = 1  # Moderate negative
        elif cumulative <= (1 - upper_threshold):
            df.iloc[dictio_['ids'], df.columns.get_loc('trend')] = 2  # Very strong negative
        else:
            df.iloc[dictio_['ids'], df.columns.get_loc('trend')] = 0  # No trend
    
    #----------------------- For Loop -----------------------#
    for idx, log_ret in enumerate(df[column]):
        sign = 1 if log_ret > 0 else -1

        if dic1 is None:  # Initialize dic1
            # print(f"\nThis one time condition 'if loop' is running \n")
            dic1 = {'ids': [idx], 'last_sign': sign, 'cumulative': (1 + log_ret)}
            continue
        last_sign = dic1['last_sign']
        if sign == last_sign and dic2 is None:  # Continue same trend
            dic1['ids'].append(idx)
            dic1['last_sign'] = sign
            dic1['cumulative'] *= (1 + log_ret)
            continue

        # 1st Reversal occuring
        if dic2 is None:  # Start dic2
            dic2 = {'ids': [idx], 'last_sign': sign, 'cumulative': (1 + log_ret)}
            continue
        last_sign = dic2['last_sign']
        if sign == last_sign and dic3 is None:  # Continue same trend
            dic2['ids'].append(idx)
            dic2['last_sign'] = sign
            dic2['cumulative'] *= (1 + log_ret)
            if len(dic2['ids']) == reverse_steps:
                assign_label(dic1, lower_threshold, upper_threshold) # Assign labels in the 'trend' column for ids of dic1
                # print(f"dic1['cumulative'] = {dic1['cumulative']}, and dic1['ids'] = {dic1['ids']}")
                dic1 = dic2
                dic2 = None
                # print(f"dic1 after trend reversal persisted and dic1 = dic2 = \n{dic1}")
                # print(f"dic2 after being reset: {dic2}\n")
            continue

        # 2nd Reversal occuring
        if dic3 is None:  # Start dic3
            dic3 = {'ids': [idx], 'last_sign': sign, 'cumulative': (1 + log_ret)}
            continue
        last_sign = dic3['last_sign']
        if sign == last_sign: # Continue same trend, there is no dic4 to check if is None
            dic3['ids'].append(idx)
            dic3['last_sign'] = sign
            dic3['cumulative'] *= (1 + log_ret)
            dic_prod = dic2['cumulative'] * dic3['cumulative']
            # if (sign == 1 and dic1['cumulative'] * dic_prod > dic1['cumulative']) or (sign == -1 and dic1['cumulative'] * dic_prod < dic1['cumulative'])):
            if (sign == 1 and dic_prod > 1) or (sign == -1 and dic_prod < 1): # More beautiful
                # Merge dic1, dic2, and dic3
                dic1['ids'] += dic2['ids'] + dic3['ids']
                dic1['last_sign'] = dic3['last_sign']
                dic1['cumulative'] *= dic2['cumulative'] * dic3['cumulative']
                dic2, dic3 = None, None
                continue

            if len(dic3['ids']) == reverse_steps:      
                assign_label(dic1, lower_threshold, upper_threshold) # Assign labels in the 'trend' column for ids of dic1
                assign_label(dic2, lower_threshold, upper_threshold) # Assign labels in the 'trend' column for ids of dic1
                dic1 = dic3
                dic2, dic3 = None, None
                # print(f"dic2 after 2nd trend reversal didn't catch up fast enough, and now \ndic1 = dic3 = {dic1}")
                # print(f"dic3 and dic2 after being reset: {dic3}\n")
            continue
            
        # 3rd Reversal occuring
        assign_label(dic1, lower_threshold, upper_threshold) # Assign labels in the 'trend' column for ids of dic1
        # Reassign values
        dic1 = dic2
        dic2 = dic3
        dic3 = {'ids': [idx], 'last_sign': sign, 'cumulative': (1 + log_ret)}
        # print(f"There was a 3rd trend reversal, and now \ndic1 = dic2 = {dic1}, \ndic2 = dic3 = {dic2}")
        # print(f"dic3 after being reset: {dic3}\n")

    # Assign remaining labels
    if dic1:
        assign_label(dic1, lower_threshold, upper_threshold)
    if dic2:
        assign_label(dic2, lower_threshold, upper_threshold)
    if dic3:
        assign_label(dic3, lower_threshold, upper_threshold)
    # print("\n#-------------------- Returning 'trend' patterns ------------------------#")
    
    # Apply filtering: Keep only selected trends, set others to 0
    df['trend'] = df['trend'].apply(lambda x: x if x in trends_to_keep else 0)

    return df

def split_X_y(sequences: list[pd.DataFrame], 
              target_column: str = 'trend',
              detect_trends_function: Callable[[pd.DataFrame, str, float, float, int, set], pd.DataFrame] = detect_trends_4, 
              column: str = 'close', 
              lower_threshold: float = 0.0009, 
              upper_threshold: float = 0.015,
              reverse_steps: int = 7,
              trends_to_keep: set = {0, 1, 2, 3, 4}) -> Tuple[np.ndarray, np.ndarray]:
    """
    Process sequences to generate features (X) and labels (y) while applying trend detection.

    Args:
    - sequences (list of pd.DataFrame): List of DataFrame sequences.
    - lower_threshold (float): Lower threshold for trend detection.
    - upper_threshold (float): Upper threshold for trend detection.
    - reverse_steps (int): Steps to reverse trends in the sequence.
    - target_column (str): Column name to use as the label (default: 'trend').
    - detect_trends_function (Callable): Function for detecting trends, defaults to `detect_trends_4`.
    - trends_to_keep (set): A set of trend categories to retain; others will be set to 0 (No Trend).

    Returns:
    - X (np.ndarray): Features array of shape (num_sequences, sequence_length, num_features).
    - y (np.ndarray): Labels array of shape (num_sequences,).
    """
    X = []
    y = []
    # count = 0
    for seq in sequences:
        # Apply trend detection on the sequence
        seq = detect_trends_function(seq, column=column, 
                                     lower_threshold=lower_threshold, 
                                     upper_threshold=upper_threshold, 
                                     reverse_steps=reverse_steps,
                                     trends_to_keep=trends_to_keep)
        # if count == 0:
        #     count = 1
        #     print(f"\nseq.head()")
        #     display(seq.head())
        #     print()
        # Extract features (X) and labels (y)
        X.append(seq.drop(columns=[target_column]).values)  # All but the target column
        y.append(seq[target_column].values)  # Target column as labels
        
    # return np.array(X, dtype=np.float32), np.array(y, dtype=np.int64)
    return np.array(X), np.array(y)

def process_and_return_splits(
    with_indicators_file_path: str,
    downsampled_data_minutes: int,
    exclude_columns: list[str],
    lower_threshold: float,
    upper_threshold: float,
    reverse_steps: int,
    sequence_length: int,
    sliding_interval: int,
    trends_to_keep: set = {0, 1, 2, 3, 4}  # Default keeps all trends
) -> tuple[
    list[list[float]],  # X_train: List of sequences, each containing a list of features
    list[list[int]],    # y_train: List of sequences, each containing a list of labels
    list[list[float]],  # X_val: List of sequences, each containing a list of features
    list[list[int]],    # y_val: List of sequences, each containing a list of labels
    list[list[float]],  # X_test: List of sequences, each containing a list of features
    list[list[int]]     # y_test: List of sequences, each containing a list of labels
]:
    """
    Processes time-series data from a CSV file and prepares it for machine learning.

    This function performs the following steps:
        1. Reads data from the specified CSV file and sorts it by date in descending order.
        2. Optionally downsamples the data to a lower frequency (e.g., 5-minute intervals).
        3. Applies Gaussian smoothing to reduce noise in the data.
        4. Calculates log returns for all numeric columns, excluding specified columns.
        5. Detects trends based on defined thresholds (`lower_threshold`, `upper_threshold`, and `reverse_steps`).
        6. Filters trends to keep only those specified in `trends_to_keep`, setting others to 0 (No Trend).
        7. Converts the processed data into sequences of a fixed length (`sequence_length`) with a sliding interval.
        8. Splits the sequences into training (80%), validation (10%), and test (10%) sets.
        9. Further splits the sequences into features (`X`) and labels (`y`) for supervised learning.

    Args:
        with_indicators_file_path (str): Path to the CSV file containing the time-series data.
        downsampled_data_minutes (int): Frequency for downsampling the data (e.g., 1 for no downsampling).
        exclude_columns (list[str]): List of column names to exclude from log return calculations.
        lower_threshold (float): Lower threshold for trend detection.
        upper_threshold (float): Upper threshold for trend detection.
        reverse_steps (int): Number of steps for reversing trends in trend detection.
        sequence_length (int): Length of sequences to create from the data.
        sliding_interval (int): Interval for sliding the window when creating sequences.
        trends_to_keep (set): A set of trend categories to retain; others will be set to 0 (No Trend). Defaults to keeping all trends {0, 1, 2, 3, 4}.

    Returns:
        tuple[list[list[float]], list[list[int]], list[list[float]], list[list[int]], list[list[float]], list[list[int]]]:
            A tuple containing:
            - X_train (list[list[float]]): Input sequences for training.
            - y_train (list[list[int]]): Target sequences for training.
            - X_val (list[list[float]]): Input sequences for validation.
            - y_val (list[list[int]]): Target sequences for validation.
            - X_test (list[list[float]]): Input sequences for testing.
            - y_test (list[list[int]]): Target sequences for testing.

    Example:
        X_train, y_train, X_val, y_val, X_test, y_test = process_and_return_splits(
            with_indicators_file_path="data.csv",
            downsampled_data_minutes=5,
            exclude_columns=["volume"],
            lower_threshold=-0.05,
            upper_threshold=0.05,
            reverse_steps=3,
            sequence_length=50,
            sliding_interval=5,
            trends_to_keep={1, 2, 3, 4}  # Only keep categorized trends, set others to 0
        )
    """

    data_retrieved = read_csv_file(with_indicators_file_path, preview_rows=0) # 190 days of data
    data_retrieved = data_retrieved.sort_index(ascending=False)

    #---------------------------------------------------------------------------------------
    if downsampled_data_minutes != 1:
        print("Downsampling the data! \n")
        data_retrieved = downsample_minute_data(data_retrieved, downsampled_data_minutes)
    #---------------------------------------------------------------------------------------

    # Get missing timestamps
    missing_timestamps = pd.date_range(
        start=data_retrieved.index.min(), # Returns smallest/earliest/oldest date
        end=data_retrieved.index.max(),
        freq='1min',  # Use 'min' for a frequency of 1 minute, '30s' for a frequency of 30 seconds
        tz=data_retrieved.index.tz,
    ).difference(data_retrieved.index)
    print(f"\ndata_retrieved - Missing timestamps time: \n{missing_timestamps}") 

    data_gaussian = gaussian_smoothing(data_retrieved, sigma=7)

    # Get missing timestamps
    missing_timestamps = pd.date_range(
        start=data_gaussian.index.min(), # Returns smallest/earliest/oldest date
        end=data_gaussian.index.max(),
        freq='1min',  # Use 'min' for a frequency of 1 minute, '30s' for a frequency of 30 seconds
        tz=data_gaussian.index.tz,
    ).difference(data_gaussian.index)
    print(f"\ndata_gaussian - Missing timestamps time: \n{missing_timestamps}\n")

    data_log_return = calculate_log_returns_all_columns(data_gaussian, exclude_columns=exclude_columns)

    # Get missing timestamps
    missing_timestamps = pd.date_range(
        start=data_log_return.index.min(), # Returns smallest/earliest/oldest date
        end=data_log_return.index.max(),
        freq='1min',  # Use 'min' for a frequency of 1 minute, '30s' for a frequency of 30 seconds
        tz=data_log_return.index.tz,
    ).difference(data_log_return.index)
    print(f"\ndata_log_return - Missing timestamps time: \n{missing_timestamps}\n") 

    # Check if there are missing timestamps
    if missing_timestamps.empty:
        print("No missing timestamps.")
    else:
        for timestamp in missing_timestamps:
            print(f"\nMissing timestamp: {timestamp}")
            
            # Create a placeholder for the missing timestamp
            if timestamp not in data_log_return.index:
                print('Missing')
            
            # Get data before and after the missing timestamp
            before_data = data_log_return[data_log_return.index < timestamp].tail(5)  # 5 data points before
            after_data = data_log_return[data_log_return.index > timestamp].head(5)  # 5 data points after
            
            # Display surrounding data
            if not before_data.empty:
                print("\nData before:")
                print(before_data)
            else:
                print("\nNo data available before the missing timestamp.")
            
            if not after_data.empty:
                print("\nData after:")
                print(after_data)
            else:
                print("\nNo data available after the missing timestamp.")

    sequences = created_sequences_2(data_log_return, sequence_length, sliding_interval)

    # Split sequences into training, validation, and test sets
    train_size = int(len(sequences) * 0.8)
    val_size = int(len(sequences) * 0.1)

    train_sequences = sequences[:train_size]
    val_sequences = sequences[train_size:train_size + val_size]
    test_sequences = sequences[train_size + val_size:]

    print(f"""
    Number of sequences:
        - sequences[0].shape: {sequences[0].shape}
        - Total sequences: {len(sequences)}
        - Train sequences: {len(train_sequences)}
        - Validation sequences: {len(val_sequences)}
        - Test sequences: {len(test_sequences)}
    """)

    # Process train, validation, and test sets
    X_train, y_train = split_X_y(train_sequences, 
                                target_column='trend',
                                detect_trends_function = detect_trends_4,
                                column= 'close',
                                lower_threshold=lower_threshold, 
                                upper_threshold=upper_threshold, 
                                reverse_steps=reverse_steps,
                                trends_to_keep=trends_to_keep)

    X_val, y_val = split_X_y(val_sequences, 
                            target_column='trend',
                            detect_trends_function = detect_trends_4,
                            column= 'close',
                            lower_threshold=lower_threshold, 
                            upper_threshold=upper_threshold, 
                            reverse_steps=reverse_steps,
                            trends_to_keep=trends_to_keep)

    X_test, y_test = split_X_y(test_sequences, 
                            target_column='trend',
                            detect_trends_function = detect_trends_4,
                            column= 'close',
                            lower_threshold=lower_threshold, 
                            upper_threshold=upper_threshold, 
                            reverse_steps=reverse_steps,
                            trends_to_keep=trends_to_keep)

    # Checking X arrays
    for idx, seq in enumerate(X_train):  # Loop through sequences
        for sub_idx, feature_set in enumerate(seq):  # Loop through data points
            for feature_idx, feature in enumerate(feature_set):  # Loop through features
                if not isinstance(feature, (float, np.float32)):  # Check each feature
                    print(f"Unexpected type in X_train at sequence {idx}, data point {sub_idx}, feature {feature_idx}: {type(feature)}")

    # Checking y arrays
    for idx, seq in enumerate(y_train):  # Loop through sequences
        for sub_idx, label in enumerate(seq):  # Loop through data points (labels)
            if not isinstance(label, (int, np.int64)):  # Check each label
                print(f"Unexpected type in y_train at sequence {idx}, data point {sub_idx}: {type(label)}")

    # Checking X arrays
    for idx, seq in enumerate(X_val):  # Loop through sequences
        for sub_idx, feature_set in enumerate(seq):  # Loop through data points
            for feature_idx, feature in enumerate(feature_set):  # Loop through features
                if not isinstance(feature, (float, np.float32)):  # Check each feature
                    print(f"Unexpected type in X_val at sequence {idx}, data point {sub_idx}, feature {feature_idx}: {type(feature)}")
    # Checking y arrays
    for idx, seq in enumerate(y_val):  # Loop through sequences
        for sub_idx, label in enumerate(seq):  # Loop through data points (labels)
            if not isinstance(label, (int, np.int64)):  # Check each label
                print(f"Unexpected type in y_val at sequence {idx}, data point {sub_idx}: {type(label)}")

    # Checking X arrays
    for idx, seq in enumerate(X_test):  # Loop through sequences
        for sub_idx, feature_set in enumerate(seq):  # Loop through data points
            for feature_idx, feature in enumerate(feature_set):  # Loop through features
                if not isinstance(feature, (float, np.float32)):  # Check each feature
                    print(f"Unexpected type in X_test at sequence {idx}, data point {sub_idx}, feature {feature_idx}: {type(feature)}")
    # Checking y arrays
    for idx, seq in enumerate(y_test):  # Loop through sequences
        for sub_idx, label in enumerate(seq):  # Loop through data points (labels)
            if not isinstance(label, (int, np.int64)):  # Check each label
                print(f"Unexpected type in y_test at sequence {idx}, data point {sub_idx}: {type(label)}")

    if isinstance(y_train, np.ndarray) and y_train.dtype == np.object_:
        # Convert to numeric if needed
        y_train = np.array(y_train, dtype=np.int64)

    if isinstance(y_val, np.ndarray) and y_val.dtype == np.object_:
        # Convert to numeric if needed
        y_val = np.array(y_val, dtype=np.int64)

    if isinstance(y_test, np.ndarray) and y_test.dtype == np.object_:
        # Convert to numeric if needed
        y_test = np.array(y_test, dtype=np.int64)

    close_col_index = data_log_return.columns.get_loc('close') # 'date' is set as index so doesnt count as a column
    Number_features = X_train.shape[-1]
    print(f"close_col_index = {close_col_index}")
    print(f"Number_features = {Number_features}")

    return X_train, y_train, X_val, y_val, X_test, y_test, Number_features

def print_class_distribution(y, var_name: str) -> None:
    """
    Prints the class distribution of a label array.

    Args:
        y: Tensor, array, or list of class labels.
        var_name: Name of the variable (for display).
    """
    if isinstance(y, torch.Tensor):
        y = y.cpu().numpy()
    flattened = np.array(y).flatten()

    unique_classes, counts = np.unique(flattened, return_counts=True)
    total = counts.sum()

    header = f"Class Distribution for '{var_name}':"
    line_parts = [
        f"Class {int(c):<3} Percent: {(count / total) * 100:>6.2f}%"
        for c, count in zip(unique_classes, counts)
    ]
    print(header.ljust(40) + " || ".join(line_parts))


## __All (Initial) parameters__

In [ ]:
ticker = 'BTC-USD'
downsampled_data_minutes = 1 # No downsampling

# Step 0 (Again): Identify parameters for trend settings of the loaded data with 1,000 data points
lower_threshold = 0.0009 
upper_threshold = 0.015  
reverse_steps = 13       

# Features not to be included in the analysis
exclude_columns= ['MACD', 'MACD_signal', 'ROC_10', 'OBV', 'AD_Line']

# Step 3, under ### Correlation Analysis
# Compute correlations with the 'trend' column
# corr = data_trends.corr()
# trend_corr = corr['trend'].sort_values(ascending=False)
strongly_correlated = ['close', 'open', 'SMA_5', 'high', 'low', 'EMA_10', 'SMA_10'] # Strongly correlated (correlation > 0.6)
moderately_correlated = ['BB_middle', 'BB_lower', 'BB_upper', 'RSI_14']             # Moderately correlated (correlation between 0.3 and 0.6)
weakly_correlated = ['SMA_50', 'volume', 'BBW', 'ATR_14']                           # Weakly correlated or negligible (correlation <~ 0.3)

# Add the weakly_correlated and moderately_correlated features to exclude_columns.
exclude_columns += weakly_correlated + moderately_correlated

sequence_length = 1000
sliding_interval = 60

## __Check GPU, CUDA, Pytorch__

### GPU details

In [ ]:
!nvidia-smi

Wed Apr  2 02:15:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.14                 Driver Version: 566.14         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   45C    P8              3W /   95W |    1944MiB /   8188MiB |     11%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### CUDA Details

In [ ]:
def check_gpu_config():
    """
    Check GPU availability and display detailed configuration information.
    """
    # Check if GPU is available
    gpu_available = torch.cuda.is_available()
    
    # Print header
    print("=" * 50)
    print("GPU Configuration Check".center(50))
    print("=" * 50)
    
    # Basic GPU availability
    print(f"{'PyTorch Version':<25}: {torch.__version__}")
    print(f"{'GPU Available':<25}: {'Yes' if gpu_available else 'No'}")
    
    # If GPU is available, print detailed info
    if gpu_available:
        print("-" * 50)
        print("GPU Details".center(50))
        print("-" * 50)
        
        # Device info
        print(f"{'Device Name':<25}: {torch.cuda.get_device_name(0)}")
        print(f"{'Number of GPUs':<25}: {torch.cuda.device_count()}")
        print(f"{'Current Device Index':<25}: {torch.cuda.current_device()}")
        
        # Compute capability and CUDA cores
        props = torch.cuda.get_device_properties(0)
        print(f"{'Compute Capability':<25}: {props.major}.{props.minor}")
        print(f"{'Total CUDA Cores':<25}: {props.multi_processor_count * 128}")  # Approx. 128 cores per SM
        
        # Memory info
        total_memory = props.total_memory / (1024 ** 3)  # Convert to GB
        memory_allocated = torch.cuda.memory_allocated(0) / (1024 ** 3)
        memory_reserved = torch.cuda.memory_reserved(0) / (1024 ** 3)
        print(f"{'Total Memory (GB)':<25}: {total_memory:.2f}")
        print(f"{'Allocated Memory (GB)':<25}: {memory_allocated:.2f}")
        print(f"{'Reserved Memory (GB)':<25}: {memory_reserved:.2f}")
    else:
        print("-" * 50)
        print("No GPU detected. Running on CPU.".center(50))
        print("-" * 50)
    
    print("=" * 50)

if __name__ == "__main__":
    check_gpu_config()

             GPU Configuration Check              
PyTorch Version          : 2.4.1+cu124
GPU Available            : Yes
--------------------------------------------------
                   GPU Details                    
--------------------------------------------------
Device Name              : NVIDIA GeForce RTX 4070
Number of GPUs           : 1
Current Device Index     : 0
Compute Capability       : 8.9
Total CUDA Cores         : 5888
Total Memory (GB)        : 11.99
Allocated Memory (GB)    : 0.00
Reserved Memory (GB)     : 0.00


### PyTorch Details

In [ ]:
def print_torch_config():
    """Print PyTorch and CUDA configuration in a formatted manner."""
    print("=" * 50)
    print("PyTorch Configuration".center(50))
    print("=" * 50)
    
    # Basic PyTorch and CUDA info
    print(f"{'PyTorch Version':<25}: {torch.__version__}")
    print(f"{'CUDA Compiled Version':<25}: {torch.version.cuda}")
    print(f"{'CUDA Available':<25}: {'Yes' if torch.cuda.is_available() else 'No'}")
    print(f"{'Number of GPUs':<25}: {torch.cuda.device_count()}")

    # GPU details if available
    if torch.cuda.is_available():
        print(f"{'GPU Name':<25}: {torch.cuda.get_device_name(0)}")

    print("-" * 50)
    
    # Seed setting
    torch.manual_seed(42)
    print(f"{'Random Seed':<25}: 42 (Seeding successful!)")
    
    print("=" * 50)

if __name__ == "__main__":
    print_torch_config()

              PyTorch Configuration               
PyTorch Version          : 2.4.1+cu124
CUDA Compiled Version    : 12.4
CUDA Available           : Yes
Number of GPUs           : 1
GPU Name                 : NVIDIA GeForce RTX 4070
--------------------------------------------------
Random Seed              : 42 (Seeding successful!)


## __Build the GRU Model__

#### Bi-Directional GRU with Attention ( _Updated to add extract_features()_ )

In [10]:
###############################
# Updated Base Model
###############################
class BiGRUWithAttention(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, output_size: int, num_layers: int, dropout: float = 0.0):
        super(BiGRUWithAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        # Bi-Directional GRU Layer
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True, dropout=dropout)
        # Attention layer
        self.attention_fc = nn.Linear(hidden_size * 2, hidden_size * 2)  # Hidden size * 2 for bi-directional
        # Fully connected layer for classification
        self.fc = nn.Linear(hidden_size * 2, output_size)
        self.dropout = nn.Dropout(dropout)  # Apply dropout before the fully connected layer
        self.init_weights()

    def init_weights(self):
        for name, param in self.named_parameters():
            if 'weight' in name:
                nn.init.xavier_uniform_(param)  # Xavier initialization for weights
            elif 'bias' in name:
                nn.init.constant_(param, 0)  # Zero initialization for biases

    def extract_features(self, x: torch.Tensor) -> torch.Tensor:
        """
        This method computes and returns the features after the GRU,
        attention and dropout layers, but before applying the final fc layer.
        """
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)  # Bi-directional: num_layers * 2
        # Bi-Directional GRU forward pass
        out, _ = self.gru(x, h0)  # out shape: (batch_size, seq_length, hidden_size * 2)
        # Attention mechanism
        attn_weights = torch.tanh(self.attention_fc(out))  # Shape: (batch_size, seq_length, hidden_size * 2)
        out = attn_weights * out  # Element-wise product for attention
        out = self.dropout(out)  # Apply dropout
        return out

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        features = self.extract_features(x)
        # Fully connected layer (applied at each time step)
        out = self.fc(features)  # Shape: (batch_size, seq_length, output_size)
        return out


#### LoRA Wrapper Model

In [11]:
#############################################
# LoRA Wrapper Model for Period 2 and Beyond
#############################################
class BiGRUWithLoRA(nn.Module):
    def __init__(self, previous_model: Union[BiGRUWithAttention, "BiGRUWithLoRA"], 
                 old_num_classes: int, new_total_classes: int, lora_rank: int = 4):
        """
        Arguments:
            previous_model: A pretrained BiGRUWithAttention (Period 1) or an already LoRA‑wrapped model (Period 2).
            old_num_classes: Number of classes learned so far.
            new_total_classes: Total number of classes for the current period.
            lora_rank: Rank for the low‑rank update.
        """
        super(BiGRUWithLoRA, self).__init__()
        # If previous_model already has a base_model attribute, use that; otherwise, use the model directly.
        if hasattr(previous_model, "base_model"):
            self.base_model = previous_model.base_model
        else:
            self.base_model = previous_model

        # Freeze all parameters of the base model.
        for param in self.base_model.parameters():
            param.requires_grad = False

        # Create a new classification layer that maps features to new_total_classes.
        in_features = self.base_model.fc.in_features
        self.fc_expanded = nn.Linear(in_features, new_total_classes)

        # Copy the old fc weights and biases from previous_model’s classification head.
        # Note: In previous_model, if it is a BiGRUWithLoRA instance then its fc layer is available as .fc_expanded
        # Otherwise, for a plain BiGRUWithAttention, use its original fc.
        with torch.no_grad():
            if hasattr(previous_model, "fc_expanded"):
                # previous_model is already a LoRA-wrapped model
                self.fc_expanded.weight[:old_num_classes, :] = previous_model.fc_expanded.weight
                self.fc_expanded.bias[:old_num_classes] = previous_model.fc_expanded.bias
            else:
                # previous_model is a base model (BiGRUWithAttention)
                self.fc_expanded.weight[:old_num_classes, :] = previous_model.fc.weight
                self.fc_expanded.bias[:old_num_classes] = previous_model.fc.bias

        # Register a hook to freeze the gradient for the non-shifting class (class 0).
        def freeze_class0_grad(grad):
            # Set the gradient of row 0 (for class 0) to zero.
            grad[0].zero_()
            return grad

        self.fc_expanded.weight.register_hook(freeze_class0_grad)
        if self.fc_expanded.bias is not None:
            # For bias, also zero out the gradient for class 0.
            self.fc_expanded.bias.register_hook(lambda grad: torch.cat([torch.zeros(1, device=grad.device), grad[1:]]))

        # Define LoRA parameters to enable a low-rank update.
        # LoRA adds an additive term: features @ lora_A.T @ lora_B.T.
        self.lora_rank = lora_rank
        self.lora_A = nn.Parameter(torch.randn(lora_rank, in_features) * 0.01)
        self.lora_B = nn.Parameter(torch.randn(new_total_classes, lora_rank) * 0.01)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Obtain features from the frozen base model.
        features = self.base_model.extract_features(x)  # shape: (batch, seq_len, feat_dim)
        batch, seq_len, feat_dim = features.shape
        features_flat = features.view(-1, feat_dim)  # collapse batch and seq_len dims
        
        # Compute the fixed (copied) classification output.
        base_logits = self.fc_expanded(features_flat)
        # Compute the LoRA contribution.
        # First: low-rank projection A
        lora_proj = features_flat @ self.lora_A.t()    # shape: (batch*seq_len, lora_rank)
        # Then: project back with B
        lora_logits = lora_proj @ self.lora_B.t()        # shape: (batch*seq_len, new_total_classes)
        
        # Total logits is the sum of the fixed and LoRA parts.
        total_logits = base_logits + lora_logits
        # Reshape back to (batch, seq_len, new_total_classes)
        total_logits = total_logits.view(batch, seq_len, -1)
        return total_logits
    

## __Training and validation function__

### Analytical Function

In [ ]:
def compute_classwise_accuracy(student_logits_flat, y_batch, class_correct, class_total):
    """
    Computes per-class accuracy by accumulating correct and total samples for each class using vectorized operations.
    
    Args:
        student_logits_flat (torch.Tensor): Model predictions (logits) in shape [batch_size * seq_len, output_size]
        y_batch (torch.Tensor): True labels in shape [batch_size * seq_len]
        class_correct (dict): Dictionary to store correct predictions per class
        class_total (dict): Dictionary to store total samples per class
    """
    # Ensure inputs are on the same device
    if student_logits_flat.device != y_batch.device:
        raise ValueError("student_logits_flat and y_batch must be on the same device")

    # Convert logits to predicted class indices
    predictions = torch.argmax(student_logits_flat, dim=-1)  # Shape: [batch_size * seq_len]

    # Compute correct predictions mask
    correct_mask = (predictions == y_batch)  # Shape: [batch_size * seq_len], boolean

    # Get unique labels in this batch
    unique_labels = torch.unique(y_batch)

    # Update class_total and class_correct using vectorized operations
    for label in unique_labels:
        label = label.item()  # Convert tensor to scalar
        if label not in class_total:
            class_total[label] = 0
            class_correct[label] = 0
        
        # Count total samples for this label
        label_mask = (y_batch == label)
        class_total[label] += label_mask.sum().item()
        
        # Count correct predictions for this label
        class_correct[label] += (label_mask & correct_mask).sum().item()

### Training and validation function

In [ ]:
# Training and validation function for all Periods.
def train_and_validate(model, output_size, criterion, optimizer, 
                       X_train, y_train, X_val, y_val, scheduler, 
                       use_scheduler=None, num_epochs=10, batch_size=64, 
                       model_saving_folder=None, model_name=None, stop_signal_file=None):
    print("'train_and_validate' function started. \n")
    # Ensure model saving folder exists (deleting existing first if there is one)
    if model_saving_folder and os.path.exists(model_saving_folder):
        # os.rmdir(model_saving_folder) # Only works on empty folders 
        shutil.rmtree(model_saving_folder) # Safely remove all contents
        if not os.path.exists(model_saving_folder):
            print(f"Existing folder has been removed : {model_saving_folder}\n")
    if model_saving_folder and not os.path.exists(model_saving_folder):
        os.makedirs(model_saving_folder)
    if not model_saving_folder:
        model_saving_folder = './saved_models'
    if not model_name:
        model_name = 'model'

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    # Convert data to tensors # Returns a copy, original is safe
    X_train = torch.tensor(X_train, dtype=torch.float32).to(device)  # (seqs, seq_len, features)
    y_train = torch.tensor(y_train, dtype=torch.long).to(device)    # (seqs, seq_len)
    X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
    y_val = torch.tensor(y_val, dtype=torch.long).to(device)
    # Create TensorDatasets
    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)
    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    print("y_train:")
    print(type(y_train))
    print(y_train.dtype)
    print(y_train.shape)
    print("X_train:")
    print(type(X_train))
    print(X_train.dtype)
    print(X_train.shape)
    print("\ny_val:")
    print(type(y_val))
    print(y_val.dtype)
    print(y_val.shape)
    print("X_val:")
    print(type(X_val))
    print(X_val.dtype)
    print(X_val.shape)

    # Debug prints for TensorDataset and DataLoader
    print("\nDataset Lengths:")
    print(f"Train Dataset Length: {len(train_dataset)}")
    print(f"Validation Dataset Length: {len(val_dataset)}")

    print("\nDataLoader Batch Sizes:")
    print(f"Number of Batches in Train DataLoader: {len(train_loader)}")
    print(f"Number of Batches in Validation DataLoader: {len(val_loader)}")

    # Additional details for y_train, y_val, and y_test
    print("\ny_train Unique Values and Stats:")
    print(f"Unique values in y_train: {y_train.unique()}")
    print(f"y_train Min: {y_train.min()}, Max: {y_train.max()}")

    print("\ny_val Unique Values and Stats:")
    print(f"Unique values in y_val: {y_val.unique()}")
    print(f"y_val Min: {y_val.min()}, Max: {y_val.max()}")

    # Device check
    print("\nDevice Info:")
    print(f"X_train Device: {X_train.device}")
    print(f"y_train Device: {y_train.device}")
    print(f"X_val Device: {X_val.device}")
    print(f"y_val Device: {y_val.device}\n")

    # Calculate number of batches
    # num_batches = (len(X_train) + batch_size - 1) // batch_size

    global best_results  # Ensure we can modify the external variable if defined outside.
    best_results = []    # Start empty each training run
    model.train()

    for epoch in range(num_epochs):
        epoch_loss = 0
        class_correct = {}  # Dictionary to store correct predictions per class
        class_total = {}  # Dictionary to store total samples per class
        if stop_signal_file and os.path.exists(stop_signal_file):
            print("\nStop signal detected. Exiting training loop safely.\n")
            break
        model.train()
        i=0
        for X_batch, y_batch in train_loader:
            # Reset gradients before forward pass
            optimizer.zero_grad()  # Best practice
            # Forward pass
            outputs = model(X_batch)
            outputs = outputs.view(-1, output_size)
            y_batch = y_batch.view(-1)

            if epoch == 1 and i < 3:
                i += 1
                print(f"\nUnique target values: {y_batch.unique()}")
                print(f"Target dtype: {y_batch.dtype}")
                print(f"Min target: {y_batch.min()}, Max target: {y_batch.max()}")
                print("Unique classes in y_train:", y_train.unique())
                print(f"Unique classes in y_val: {y_val.unique()}\n")
            
            # Compute loss
            loss = criterion(outputs, y_batch)
            # Compute class-wise accuracy (Accumulates values in dict)
            compute_classwise_accuracy(outputs, y_batch, class_correct, class_total)
            # Backward pass and optimization
            # No longer reset gradients here: optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * X_batch.size(0)  # Scale back to total loss
        train_loss = epoch_loss / len(train_loader.dataset)

        # Compute per-class training accuracy
        # train_classwise_accuracy = {int(c): (class_correct[c] / class_total[c]) * 100 if class_total[c] > 0 else 0 
        #                            for c in sorted(class_total.keys())}
        train_classwise_accuracy = {int(c): f"{(class_correct[c] / class_total[c]) * 100:.2f}%" if class_total[c] > 0 else "0.00%" 
                                    for c in sorted(class_total.keys())}
        
        # Perform validation at the end of each epoch
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        val_class_correct = {}
        val_class_total = {}
        with torch.no_grad():
            for X_val_batch, y_val_batch in val_loader:
                val_outputs = model(X_val_batch).view(-1, output_size)
                val_labels = y_val_batch.view(-1)
                val_loss += criterion(val_outputs, val_labels).item() * X_val_batch.size(0)  # Scale to total loss
                val_predictions = torch.argmax(val_outputs, dim=-1)
                val_correct += (val_predictions == val_labels).sum().item()
                val_total += val_labels.size(0)
                # Compute per-class validation accuracy
                compute_classwise_accuracy(val_outputs, val_labels, val_class_correct, val_class_total)
        val_loss /= len(val_loader.dataset)
        val_accuracy = val_correct / val_total

        # Compute per-class validation accuracy
        # val_classwise_accuracy = {int(c): (val_class_correct[c] / val_class_total[c]) * 100 if val_class_total[c] > 0 else 0 
        #                          for c in sorted(val_class_total.keys())}
        val_classwise_accuracy = {int(c): f"{(val_class_correct[c] / val_class_total[c]) * 100:.2f}%" if val_class_total[c] > 0 else "0.00%" 
                                  for c in sorted(val_class_total.keys())}

        print(f"Epoch {epoch+1}/{num_epochs}, "
              f"Train Loss: {train_loss:.9f}, "
              f"Train-Class-Acc: {train_classwise_accuracy}, "
              f"Val Loss: {val_loss:.9f}, "
              f"Val Accuracy: {val_accuracy * 100:.2f}%, "
              f"Val-Class-Acc: {val_classwise_accuracy}, "
              f"LR: {optimizer.param_groups[0]['lr']:.9f}")

        # Save current model and update best results if applicable
        current_epoch_info = {
            "epoch": epoch+1,
            "train_loss": train_loss,
            "train_classwise_accuracy": train_classwise_accuracy,
            "val_loss": val_loss,
            "val_accuracy": val_accuracy,
            "val_classwise_accuracy": val_classwise_accuracy,
            'learning_rate': optimizer.param_groups[0]['lr'], # Optimizer state
            "model_path": os.path.join(model_saving_folder, f"{model_name}_epoch_{epoch+1}.pth")
        }

        # Insert this epoch if we have fewer than 5 results
        # or if it beats the lowest of the top 5
        if len(best_results) < 5 or val_accuracy > best_results[-1]["val_accuracy"]:
            if len(best_results) == 5:
                # Remove the worst model from the list, the last (lowest accuracy)
                worst = best_results.pop() 
                if os.path.exists(worst["model_path"]):
                    os.remove(worst["model_path"])
                    print(f"Removed old model with accuracy {worst['val_accuracy']*100:.2f}%, and file was at {worst['model_path']}")
            # Just insert and sort by val_accuracy descending
            best_results.append(current_epoch_info) 
            best_results.sort(key=lambda x: x["val_accuracy"], reverse=True)
            torch.save({ # Save this model
                'epoch': epoch+1,  # Save the current epoch
                'train_loss': train_loss,
                'val_loss': val_loss,
                'model_state_dict': model.state_dict(),  # Model weights
                'optimizer_state_dict': optimizer.state_dict(),  # Optimizer state
                'learning_rate': optimizer.param_groups[0]['lr'] # Optimizer state
            }, current_epoch_info["model_path"])
            print(f"Model saved after epoch {epoch+1} to {current_epoch_info['model_path']} \n")

        if use_scheduler == True:
            # Scheduler step should follow after considering the results (placed after otallher losses)
            scheduler.step(val_loss)


    # Save the final model
    if current_epoch_info:
        final_model_path = os.path.join(model_saving_folder, f"{model_name}_final.pth")
        torch.save({ # Save this model
            'epoch': epoch+1,  # Save the current epoch
            'train_loss': train_loss,
            'val_loss': val_loss,
            'model_state_dict': model.state_dict(),  # Model weights
            'optimizer_state_dict': optimizer.state_dict(),  # Optimizer state
            'learning_rate': optimizer.param_groups[0]['lr'] # Optimizer state
        }, final_model_path)
        print(f"\nFinal model saved to {final_model_path}")

    print("\nTraining complete. \n\nTop 5 Models Sorted by Validation Accuracy: ")
    for res in best_results:        
        print(f"Epoch {res['epoch']}/{num_epochs}, "
              f"Train Loss: {res['train_loss']:.9f}, "
              f"Train-Class-Acc: {train_classwise_accuracy}, " 
              f"Val Loss: {res['val_loss']:.9f}, "
              f"Val Accuracy: {res['val_accuracy'] * 100:.2f}%, "
              f"Val-Class-Acc: {val_classwise_accuracy}, "
              f"Model Path: {res['model_path']}")
    print('\n')
    
    del X_train, y_train, X_val, y_val, train_dataset, val_dataset, train_loader, val_loader
    torch.cuda.empty_cache()

    # Load the checkpoint
    # checkpoint = torch.load("path/to/model_checkpoint.pth")
    # # Restore model state
    # model.load_state_dict(checkpoint['model_state_dict'])
    # # Restore optimizer state
    # optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    # # Restore scheduler state (if used)
    # scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    # # Restore epoch and other metadata
    # start_epoch = checkpoint['epoch'] + 1  # Resume from the next epoch
    # loss = checkpoint['loss']  # Optional
    # print(f"Checkpoint loaded. Resuming from epoch {start_epoch}")

## __Setup before training__

### Define list_period_files_full_path

In [ ]:
def setup_file_paths(pair='BTCUSD', base_dir='Data', days=190):
    """
    Set up file paths for cryptocurrency data across multiple periods.

    Args:
        pair (str): Trading pair (e.g., 'BTCUSD').
        base_dir (str): Base directory for data storage (default 'Data').
        days (int): Number of days for each period (default 190).

    Returns:
        tuple: (base_folder_path, with_indicators_file_path, list_period_files_full_path)
    """
    # Define base file name and folder structure
    file_name = f"Polygon_{pair}_4Y_1min"
    base_folder_path = os.path.normpath(os.path.join(base_dir, file_name))
    
    # Check if folder exists
    if not os.path.isdir(base_folder_path):
        raise FileNotFoundError(f"Directory '{base_folder_path}' does not exist.")

    # Define file path with indicators for Period 1
    with_indicators_file_path = os.path.normpath(
        os.path.join(base_folder_path, f"_{file_name}_{days}_days_with_indicators.csv")
    )

    # Define file paths for all periods
    list_period_files_full_path = [
        # Period 1
        with_indicators_file_path,
        # Period 2: 2020-11-11 to 2021-05-20
        os.path.normpath(os.path.join(
            base_folder_path, f"{file_name}_{days}_days__2020-11-11__2021-05-20__with_indicators.csv"
        )),
        # Period 3: 2021-05-20 to 2021-11-26
        os.path.normpath(os.path.join(
            base_folder_path, f"{file_name}_{days}_days__2021-05-20__2021-11-26__with_indicators.csv"
        )),
        # Period 4: 2021-11-26 to 2022-06-04
        os.path.normpath(os.path.join(
            base_folder_path, f"{file_name}_{days}_days__2021-11-26__2022-06-04__with_indicators.csv"
        )),
        # Period 5: 2022-06-04 to 2022-12-11
        os.path.normpath(os.path.join(
            base_folder_path, f"{file_name}_{days}_days__2022-06-04__2022-12-11__with_indicators.csv"
        )),
    ]

    return base_folder_path, with_indicators_file_path, list_period_files_full_path

def print_folder_contents(folder_path):
    """Print all files in the specified folder."""
    print("\n📂 Folder Contents:")
    for file in os.listdir(folder_path):
        print(f"Found file: {file}")

if __name__ == "__main__":
    # Set up paths
    base_folder_path, with_indicators_file_path, list_period_files_full_path = setup_file_paths()

    # Print results
    print("=" * 70)
    print("File Path Configuration".center(70))
    print("=" * 70)
    
    print(f"{'Base Folder Path':<25}: {base_folder_path}")
    print(f"{'Period 1 File Path':<25}: {with_indicators_file_path}")
    print("-" * 70)
    
    print("List of Period Files:")
    for i, path in enumerate(list_period_files_full_path, 1):
        print(f"{'Period ' + str(i):<25}: {path}")
    
    print("=" * 70)

    # Print folder contents
    print_folder_contents(base_folder_path)

                       File Path Configuration                        
Base Folder Path         : Data\Polygon_BTCUSD_4Y_1min
Period 1 File Path       : Data\Polygon_BTCUSD_4Y_1min\_Polygon_BTCUSD_4Y_1min_190_days_with_indicators.csv
----------------------------------------------------------------------
List of Period Files:
Period 1                 : Data\Polygon_BTCUSD_4Y_1min\_Polygon_BTCUSD_4Y_1min_190_days_with_indicators.csv
Period 2                 : Data\Polygon_BTCUSD_4Y_1min\Polygon_BTCUSD_4Y_1min_190_days__2020-11-11__2021-05-20__with_indicators.csv
Period 3                 : Data\Polygon_BTCUSD_4Y_1min\Polygon_BTCUSD_4Y_1min_190_days__2021-05-20__2021-11-26__with_indicators.csv
Period 4                 : Data\Polygon_BTCUSD_4Y_1min\Polygon_BTCUSD_4Y_1min_190_days__2021-11-26__2022-06-04__with_indicators.csv
Period 5                 : Data\Polygon_BTCUSD_4Y_1min\Polygon_BTCUSD_4Y_1min_190_days__2022-06-04__2022-12-11__with_indicators.csv

📂 Folder Contents:
Found file: Polyg

### __All periods data__
'trend': Categorized trend values based on the detected phases:
- 0: No trend
- 1: Moderate negative trend
- 2: Very strong negative trend
- 3: Moderate positive trend
- 4: Very strong positive trend


## __Train the Model__

### Period 1 --> Training and saving in __*'1st_try'*__ (BiGRUWithAttention, num_layers = 4) ---> Val acc = 98.35 %
### Val-Class-Acc: {0: '98.10%', 1: '97.81%'}
```
Class Distribution for 'y_val':

Class 0   Percent:  60.61% ||
Class 1   Percent:  39.39% 
```

In [21]:
"""
- 'trend': Categorized trend values based on the detected phases:
    - 0: No trend
    - 1: Moderate negative trend
    - 2: Very strong negative trend
    - 3: Moderate positive trend
    - 4: Very strong positive trend
"""
with contextlib.redirect_stdout(open(os.devnull, 'w')):
    X_train, y_train, X_val, y_val, X_test, y_test, Number_features = process_and_return_splits(
        with_indicators_file_path = list_period_files_full_path[0], # Change 
        downsampled_data_minutes = downsampled_data_minutes,
        exclude_columns = exclude_columns,
        lower_threshold = lower_threshold,
        upper_threshold = upper_threshold,
        reverse_steps = reverse_steps,
        sequence_length = sequence_length,
        sliding_interval = sliding_interval,
        trends_to_keep = {0, 1}  # Default keeps all trends : {0, 1, 2, 3, 4}
        # trends_to_keep = {0, 1, 2, 3, 4}  # Default keeps all trends
    )

print(f"\nNumber_features = {Number_features}")

unique_classes = np.unique(y_val)
num_classes = len(unique_classes)
print(f"unique_classes = {unique_classes}")
print(f"num_classes = {num_classes}")

print_class_distribution(y_train, "y_train")
print_class_distribution(y_val, "y_val")
print_class_distribution(y_test, "y_test")



Number_features = 7
unique_classes = [0 1]
num_classes = 2
Class Distribution for 'y_train':       Class 0   Percent:  63.08% || Class 1   Percent:  36.92%
Class Distribution for 'y_val':         Class 0   Percent:  60.61% || Class 1   Percent:  39.39%
Class Distribution for 'y_test':        Class 0   Percent:  58.04% || Class 1   Percent:  41.96%


In [22]:
for var in ["X_train", "y_train", "X_val", "y_val", "X_test", "y_test", "Number_features", "unique_classes", "num_classes"]:
    if var in locals():
        del locals()[var]

In [61]:
#----------------------------------------------------------------------
# Initialize the list to store results across runs
track_across_runs = []
#----------------------------------------------------------------------

# Model parameters
input_size = Number_features  # Number of features
hidden_size = 64  # Number of GRU units
output_size = num_classes # Must be dynamic, up to 5  # Number of trend classes (0, 15, 25, -15, -25)
num_layers = 4  # Number of GRU layers
num_epochs= 2000 # Number of epochs/ go through entire data
batch_size= 64 # How many sequences passed at once to the model
model_name = 'BiGRUWithAttention' # Name of the model to use for saving
best_results = [] # Initialize this outside the training function or at the beginning of training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Define a global stop signal
stop_signal_file = os.path.normpath(os.path.join('Class_Incremental_CL', 'Classif_Bi_Dir_GRU_Model/stop_training.txt'))  # Create this file to stop training
model_saving_folder = os.path.normpath(os.path.join('Class_Incremental_CL', "Classif_Bi_Dir_GRU_Model/Trained_models/Baseline/Period_1/1st_try"))
ensure_folder(model_saving_folder)

# Instantiate the model
class_gru_model = BiGRUWithAttention(input_size, hidden_size, output_size, num_layers).to(device)

# Define the loss function, optimizer and scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(class_gru_model.parameters(), lr=0.0001) # lr=0.00005
# optimizer = optim.Adam(class_gru_model.parameters(), lr=0.001, weight_decay=1e-5)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

# train_and_validate(class_gru_model, output_size, criterion, optimizer, X_train_all, y_train_all, X_val_all, y_val_all, scheduler, 
train_and_validate(class_gru_model, output_size, criterion, optimizer, X_train, y_train, X_val, y_val, scheduler, 
                   False, num_epochs, batch_size, model_saving_folder, model_name, stop_signal_file)

#----------------------------------------------------------------------
# Append only the best result (already at index 0)
track_across_runs.append(best_results[0])
#----------------------------------------------------------------------

for res in best_results:        
    print(f"Epoch {res['epoch']}/{num_epochs}, "
            f"Train Loss: {res['train_loss']:.4f}, " 
            f"Val Loss: {res['val_loss']:.4f}, "
            f"Val Accuracy: {res['val_accuracy'] * 100:.2f}%, "
            f"Model Path: {res['model_path']}")      
print(f"\nclass_gru_model: \n{class_gru_model}")

print(f"\nunique_classes = {unique_classes}")
print(f"num_classes = {num_classes}")
# del unique_classes, num_classes
for var in ["X_train", "y_train", "X_val", "y_val", "X_test", "y_test", "Number_features", "unique_classes", "num_classes"]:
    if var in locals():
        del locals()[var]


'train_and_validate' function started. 

Existing folder has been removed : Class_Incremental_CL\Classif_Bi_Dir_GRU_Model\Trained_models\Baseline\Period_1\1st_try

y_train:
<class 'torch.Tensor'>
torch.int64
torch.Size([3634, 1000])
X_train:
<class 'torch.Tensor'>
torch.float32
torch.Size([3634, 1000, 7])

y_val:
<class 'torch.Tensor'>
torch.int64
torch.Size([454, 1000])
X_val:
<class 'torch.Tensor'>
torch.float32
torch.Size([454, 1000, 7])

Dataset Lengths:
Train Dataset Length: 3634
Validation Dataset Length: 454

DataLoader Batch Sizes:
Number of Batches in Train DataLoader: 57
Number of Batches in Validation DataLoader: 8

y_train Unique Values and Stats:
Unique values in y_train: tensor([0, 1], device='cuda:0')
y_train Min: 0, Max: 1

y_val Unique Values and Stats:
Unique values in y_val: tensor([0, 1], device='cuda:0')
y_val Min: 0, Max: 1

Device Info:
X_train Device: cuda:0
y_train Device: cuda:0
X_val Device: cuda:0
y_val Device: cuda:0

Epoch 1/2000, Train Loss: 0.681195149, 

### Period 1 --> Training and saving in __*'2nd_try'*__ (BiGRUWithAttention, num_layers = 3) ---> Val acc = 98.31 %

In [18]:
"""
- 'trend': Categorized trend values based on the detected phases:
    - 0: No trend
    - 1: Moderate negative trend
    - 2: Very strong negative trend
    - 3: Moderate positive trend
    - 4: Very strong positive trend
"""
with contextlib.redirect_stdout(open(os.devnull, 'w')):
    X_train, y_train, X_val, y_val, X_test, y_test, Number_features = process_and_return_splits(
        with_indicators_file_path = list_period_files_full_path[0], # Change 
        downsampled_data_minutes = downsampled_data_minutes,
        exclude_columns = exclude_columns,
        lower_threshold = lower_threshold,
        upper_threshold = upper_threshold,
        reverse_steps = reverse_steps,
        sequence_length = sequence_length,
        sliding_interval = sliding_interval,
        trends_to_keep = {0, 1}  # Default keeps all trends : {0, 1, 2, 3, 4}
        # trends_to_keep = {0, 1, 2, 3, 4}  # Default keeps all trends
    )

print(f"\nNumber_features = {Number_features}")

unique_classes = np.unique(y_val)
num_classes = len(unique_classes)
print(f"unique_classes = {unique_classes}")
print(f"num_classes = {num_classes}")

print_class_distribution(y_train, "y_train")
print_class_distribution(y_val, "y_val")
print_class_distribution(y_test, "y_test")


Number_features = 7
unique_classes = [0 1]
num_classes = 2
Class Distribution for 'y_train':       Class 0   Percent:  63.08% || Class 1   Percent:  36.92%
Class Distribution for 'y_val':         Class 0   Percent:  60.61% || Class 1   Percent:  39.39%
Class Distribution for 'y_test':        Class 0   Percent:  58.04% || Class 1   Percent:  41.96%


In [ ]:
# Model parameters
input_size = Number_features  # Number of features
hidden_size = 64  # Number of GRU units
output_size = num_classes # Must be dynamic, up to 5  # Number of trend classes (0, 15, 25, -15, -25)
num_layers = 3  # Number of GRU layers
num_epochs= 2000 # Number of epochs/ go through entire data
batch_size= 64 # How many sequences passed at once to the model
model_name = 'BiGRUWithAttention' # Name of the model to use for saving
best_results = [] # Initialize this outside the training function or at the beginning of training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Define a global stop signal
stop_signal_file = os.path.normpath(os.path.join('Class_Incremental_CL', 'Classif_Bi_Dir_GRU_Model/stop_training.txt'))  # Create this file to stop training
model_saving_folder = os.path.normpath(os.path.join('Class_Incremental_CL', "Classif_Bi_Dir_GRU_Model/Trained_models/Baseline/Period_1/2nd_try"))
ensure_folder(model_saving_folder)

# Instantiate the model
class_gru_model = BiGRUWithAttention(input_size, hidden_size, output_size, num_layers).to(device)

# Define the loss function, optimizer and scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(class_gru_model.parameters(), lr=0.0001) # lr=0.00005
# optimizer = optim.Adam(class_gru_model.parameters(), lr=0.001, weight_decay=1e-5)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

# train_and_validate(class_gru_model, output_size, criterion, optimizer, X_train_all, y_train_all, X_val_all, y_val_all, scheduler, 
train_and_validate(class_gru_model, output_size, criterion, optimizer, X_train, y_train, X_val, y_val, scheduler, 
                   False, num_epochs, batch_size, model_saving_folder, model_name, stop_signal_file)

for res in best_results:        
    print(f"Epoch {res['epoch']}/{num_epochs}, "
            f"Train Loss: {res['train_loss']:.4f}, " 
            f"Val Loss: {res['val_loss']:.4f}, "
            f"Val Accuracy: {res['val_accuracy'] * 100:.2f}%, "
            f"Model Path: {res['model_path']}")      
print(f"\nclass_gru_model: \n{class_gru_model}")

print(f"\nunique_classes = {unique_classes}")
print(f"num_classes = {num_classes}")
# del unique_classes, num_classes
for var in ["X_train", "y_train", "X_val", "y_val", "X_test", "y_test", "Number_features", "unique_classes", "num_classes"]:
    if var in locals():
        del locals()[var]


'train_and_validate' function started. 

Existing folder has been removed : Class_Incremental_CL\Classif_Bi_Dir_GRU_Model\Trained_models\2nd_try

y_train:
<class 'torch.Tensor'>
torch.int64
torch.Size([3634, 1000])
X_train:
<class 'torch.Tensor'>
torch.float32
torch.Size([3634, 1000, 7])

y_val:
<class 'torch.Tensor'>
torch.int64
torch.Size([454, 1000])
X_val:
<class 'torch.Tensor'>
torch.float32
torch.Size([454, 1000, 7])

Dataset Lengths:
Train Dataset Length: 3634
Validation Dataset Length: 454

DataLoader Batch Sizes:
Number of Batches in Train DataLoader: 57
Number of Batches in Validation DataLoader: 8

y_train Unique Values and Stats:
Unique values in y_train: tensor([0, 1], device='cuda:0')
y_train Min: 0, Max: 1

y_val Unique Values and Stats:
Unique values in y_val: tensor([0, 1], device='cuda:0')
y_val Min: 0, Max: 1

Device Info:
X_train Device: cuda:0
y_train Device: cuda:0
X_val Device: cuda:0
y_val Device: cuda:0

Epoch 1/2000, Train Loss: 0.010724814, Val Loss: 0.011819

### Period 2 --> Training and saving in __*'1st_try'*__ (BiGRUWithLoRA, lora_r=4) ---> Val acc = 96.22 %
### Val-Class-Acc: {0: '99.28%', 1: '95.88%', 2: '81.71%'}
```
Class Distribution for 'y_val':

Class 0   Percent:  52.72% ||
Class 1   Percent:  36.38% ||
Class 2   Percent:  10.90% 
```

In [118]:
"""
- 'trend': Categorized trend values based on the detected phases:
    - 0: No trend
    - 1: Moderate negative trend
    - 2: Very strong negative trend
    - 3: Moderate positive trend
    - 4: Very strong positive trend
"""
with contextlib.redirect_stdout(open(os.devnull, 'w')):
    X_train, y_train, X_val, y_val, X_test, y_test, Number_features = process_and_return_splits(
        with_indicators_file_path = list_period_files_full_path[1], # Change 
        downsampled_data_minutes = downsampled_data_minutes,
        exclude_columns = exclude_columns,
        lower_threshold = lower_threshold,
        upper_threshold = upper_threshold,
        reverse_steps = reverse_steps,
        sequence_length = sequence_length,
        sliding_interval = sliding_interval,
        trends_to_keep = {0, 1, 2}  # Default keeps all trends : {0, 1, 2, 3, 4}
        # trends_to_keep = {0, 1, 2, 3, 4}  # Default keeps all trends
    )

print(f"\nNumber_features = {Number_features}")

unique_classes = np.unique(y_val)
num_classes = len(unique_classes)
print(f"unique_classes = {unique_classes}")
print(f"num_classes = {num_classes}")

print_class_distribution(y_train, "y_train")
print_class_distribution(y_val, "y_val")
print_class_distribution(y_test, "y_test")



Number_features = 7
unique_classes = [0 1 2]
num_classes = 3
Class Distribution for 'y_train':       Class 0   Percent:  53.41% || Class 1   Percent:  36.03% || Class 2   Percent:  10.55%
Class Distribution for 'y_val':         Class 0   Percent:  52.72% || Class 1   Percent:  36.38% || Class 2   Percent:  10.90%
Class Distribution for 'y_test':        Class 0   Percent:  49.11% || Class 1   Percent:  32.27% || Class 2   Percent:  18.62%


In [60]:
# with contextlib.redirect_stdout(open(os.devnull, 'w')):
#     X_train, y_train, X_val, y_val, X_test, y_test, Number_features = process_and_return_splits(
#         with_indicators_file_path = list_period_files_full_path[1], # Change 
#         downsampled_data_minutes = downsampled_data_minutes,
#         exclude_columns = exclude_columns,
#         lower_threshold = lower_threshold,
#         upper_threshold = upper_threshold,
#         reverse_steps = reverse_steps,
#         sequence_length = sequence_length,
#         sliding_interval = sliding_interval,
#         trends_to_keep = {0, 1, 2}  # Default keeps all trends : {0, 1, 2, 3, 4}
#         # trends_to_keep = {0, 1, 2, 3, 4}  # Default keeps all trends
#     )
# print(f"\nNumber_features = {Number_features}")
# unique_classes = np.unique(y_val)
# num_classes = len(unique_classes)
# print(f"unique_classes = {unique_classes}")
# print(f"num_classes = {num_classes}")

# Model parameters
input_size = Number_features  # Number of features
hidden_size = 64  # Number of GRU units
output_size = num_classes # Must be dynamic, up to 5  # Number of trend classes (0, 15, 25, -15, -25)
num_layers = 4  # Number of GRU layers
dropout = 0.0
lora_r = 4 # Rank of the low-rank update matrices
learning_rate = 0.0001
num_epochs= 500 # Number of epochs/ go through entire data
batch_size= 64 # How many sequences passed at once to the model
model_name = 'BiGRUWithAttentionLoRA' # Name of the model to use for saving
best_results = [] # Initialize this outside the training function or at the beginning of training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Instantiate the previous model
previous_model = BiGRUWithAttention(input_size, hidden_size, output_size - 1, num_layers)#.to(device)
best_model_path = r"Class_Incremental_CL\Classif_Bi_Dir_GRU_Model\Trained_models\Baseline\Period_1\1st_try\BiGRUWithAttention_epoch_1987.pth"
previous_model_checkpoint_path = os.path.normpath(best_model_path)
previous_model_checkpoint = torch.load(previous_model_checkpoint_path, map_location=device, weights_only=True)
previous_model.load_state_dict(previous_model_checkpoint['model_state_dict'])
del previous_model_checkpoint
gc.collect()
print(f"Loaded previous model from: \n\t{previous_model_checkpoint_path}")
print(f"\n{previous_model}\n")

# Convert data to tensors # Returns a copy, original is safe
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)  # (seqs, seq_len, features)
y_train = torch.tensor(y_train, dtype=torch.long).to(device)    # (seqs, seq_len)
X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val = torch.tensor(y_val, dtype=torch.long).to(device)
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

previous_model.to(device)
previous_model.eval()
val_loss = 0.0
val_correct = 0
val_total = 0
val_class_correct = {}
val_class_total = {}
with torch.no_grad():
    for X_val_batch, y_val_batch in val_loader:
        val_outputs = previous_model(X_val_batch).view(-1, output_size-1)

        # # Get raw model predictions (originally only 2 logits)
        # val_outputs = previous_model(X_val_batch)  # Shape: [batch_size, seq_len, 2]
        # # Add a 3rd logit with value 0.0 for all samples
        # third_logit = torch.zeros((*val_outputs.shape[:-1], 1), device=val_outputs.device)  # Shape: [batch_size, seq_len, 1]
        # val_outputs = torch.cat((val_outputs, third_logit), dim=-1)  # Shape: [batch_size, seq_len, 3]
        # # Now apply view operation to match loss function's expected shape
        # val_outputs = val_outputs.view(-1, output_size)  # Correct shape: [batch_size * seq_len, 3]

        val_labels = y_val_batch.view(-1)
        # val_loss += criterion(val_outputs, val_labels).item() * X_val_batch.size(0)  # Scale to total loss
        # val_predictions = torch.argmax(val_outputs, dim=-1)
        # val_correct += (val_predictions == val_labels).sum().item()
        # val_total += val_labels.size(0)
        # Compute per-class validation accuracy
        compute_classwise_accuracy(val_outputs, val_labels, val_class_correct, val_class_total)
# val_loss /= len(val_loader.dataset)
# val_accuracy = val_correct / val_total
# print(f"val_accuracy = {val_accuracy* 100:.2f}%")

# Compute per-class validation accuracy
# val_classwise_accuracy = {int(c): (val_class_correct[c] / val_class_total[c]) * 100 if val_class_total[c] > 0 else 0 
#                          for c in sorted(val_class_total.keys())}
val_classwise_accuracy = {int(c): f"{(val_class_correct[c] / val_class_total[c]) * 100:.2f}%" if val_class_total[c] > 0 else "0.00%" 
                            for c in sorted(val_class_total.keys())}

print(f"\nval_classwise_accuracy: {val_classwise_accuracy}")
print(f"val_class_correct: {val_class_correct}")
print(f"val_class_total: {val_class_total}")


del X_train, y_train, X_val, y_val, train_dataset, val_dataset, train_loader, val_loader, val_classwise_accuracy
torch.cuda.empty_cache()

for var in ["X_train", "y_train", "X_val", "y_val", "X_test", "y_test", "Number_features", "unique_classes", "num_classes"]:
    if var in locals():
        del locals()[var]

Loaded previous model from: 
	Class_Incremental_CL\Classif_Bi_Dir_GRU_Model\Trained_models\Baseline\Period_1\1st_try\BiGRUWithAttention_epoch_1987.pth

BiGRUWithAttention(
  (gru): GRU(7, 64, num_layers=4, batch_first=True, bidirectional=True)
  (attention_fc): Linear(in_features=128, out_features=128, bias=True)
  (fc): Linear(in_features=128, out_features=2, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
)


val_classwise_accuracy: {0: '99.54%', 1: '95.88%', 2: '0.00%'}
val_class_correct: {0: 238248, 1: 158370, 2: 0}
val_class_total: {0: 239337, 1: 165167, 2: 49496}


In [119]:
# Model parameters
input_size = Number_features  # Number of features
hidden_size = 64  # Number of GRU units
output_size = num_classes # Must be dynamic, up to 5  # Number of trend classes (0, 15, 25, -15, -25)
num_layers = 4  # Number of GRU layers
dropout = 0.0
learning_rate = 0.0001
weight_decay=1e-5 # For the optimizer
use_scheduler = False
lora_r = 4 # Rank of the low-rank update matrices
# ---- ---- ---- #
num_epochs= 2000 # Number of epochs/ go through entire data
batch_size= 64 # How many sequences passed at once to the model
model_name = 'BiGRUWithLoRA' # Name of the model to use for saving
best_results = [] # Initialize this outside the training function or at the beginning of training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Define a global stop signal
stop_signal_file = os.path.normpath(os.path.join('Class_Incremental_CL', 'Classif_Bi_Dir_GRU_Model/stop_training.txt'))  # Create this file to stop training
model_saving_folder = os.path.normpath(os.path.join('Class_Incremental_CL', "Classif_Bi_Dir_GRU_Model/Trained_models/LoRA/Period_2/1st_try"))
ensure_folder(model_saving_folder)

#--------------------------------
# Instantiate the previous model
#--------------------------------
previous_model = BiGRUWithAttention(input_size, hidden_size, output_size - 1, num_layers, dropout=dropout)
#-------------------------------------------------------------------------
best_model_path = r"Class_Incremental_CL\Classif_Bi_Dir_GRU_Model\Trained_models\Baseline\Period_1\1st_try\BiGRUWithAttention_epoch_1987.pth"
#----------------------------------------------------------------------
# Initialize the list to store results across runs
track_across_runs = []
#----------------------------------------------------------------------
#-------------------------------------------------------------------------
previous_model_checkpoint_path = os.path.normpath(best_model_path)
previous_model_checkpoint = torch.load(previous_model_checkpoint_path, map_location=torch.device('cpu'), weights_only=True)
previous_model.load_state_dict(previous_model_checkpoint['model_state_dict'])
print(f"Loaded previous model from: \n\t{previous_model_checkpoint_path}")
print(f"\n{previous_model}\n")
del previous_model_checkpoint, previous_model_checkpoint_path
gc.collect()

#-------------------------------------------------------------------------
# Instantiate the current model
current_model = BiGRUWithLoRA(previous_model, old_num_classes=output_size-1, new_total_classes=output_size, lora_rank=lora_r).to(device)
#-------------------------------------------------------------------------

# Define the loss function, optimizer and scheduler
criterion = nn.CrossEntropyLoss()
# Only parameters with requires_grad=True (the LoRA adapters) will be updated.
# Only parameters of the new head (fc_expanded, lora_A, lora_B) are trainable.
optimizer = optim.Adam(filter(lambda p: p.requires_grad, 
                              current_model.parameters()), 
                              lr=learning_rate, 
                              weight_decay=weight_decay)
# optimizer = optim.Adam(class_gru_model.parameters(), lr=0.001, weight_decay=1e-5)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

train_and_validate(
    model=current_model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val,
    scheduler=scheduler,
    use_scheduler=use_scheduler,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name, # Use base name for saved files in this period
    stop_signal_file=stop_signal_file
)

#----------------------------------------------------------------------
# Append only the best result (already at index 0)
track_across_runs.append(best_results[0])
#----------------------------------------------------------------------

for res in best_results:        
    print(f"Epoch {res['epoch']}/{num_epochs}, "
            f"Train Loss: {res['train_loss']:.4f}, " 
            f"Val Loss: {res['val_loss']:.4f}, "
            f"Val Accuracy: {res['val_accuracy'] * 100:.2f}%, "
            f"Model Path: {res['model_path']}")      
print(f"\ncurrent_model: \n{current_model}")

print(f"\nunique_classes = {unique_classes}")
print(f"num_classes = {num_classes}")

for var in [
    "X_train", "y_train", "X_val", "y_val", "X_test", "y_test",
    "Number_features", "unique_classes", "num_classes",
    "previous_model", "current_model",
    "criterion", "optimizer", "scheduler", 
]:
    if var in locals():
        del locals()[var]
# --- Force garbage collection ---
torch.cuda.empty_cache()
gc.collect()


Loaded previous model from: 
	Class_Incremental_CL\Classif_Bi_Dir_GRU_Model\Trained_models\Baseline\Period_1\1st_try\BiGRUWithAttention_epoch_1987.pth

BiGRUWithAttention(
  (gru): GRU(7, 64, num_layers=4, batch_first=True, bidirectional=True)
  (attention_fc): Linear(in_features=128, out_features=128, bias=True)
  (fc): Linear(in_features=128, out_features=2, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
)

'train_and_validate' function started. 

Existing folder has been removed : Class_Incremental_CL\Classif_Bi_Dir_GRU_Model\Trained_models\LoRA\Period_2\1st_try

y_train:
<class 'torch.Tensor'>
torch.int64
torch.Size([3634, 1000])
X_train:
<class 'torch.Tensor'>
torch.float32
torch.Size([3634, 1000, 7])

y_val:
<class 'torch.Tensor'>
torch.int64
torch.Size([454, 1000])
X_val:
<class 'torch.Tensor'>
torch.float32
torch.Size([454, 1000, 7])

Dataset Lengths:
Train Dataset Length: 3634
Validation Dataset Length: 454

DataLoader Batch Sizes:
Number of Batches in Train DataLoader:

0

In [120]:
print(best_model_path)

Class_Incremental_CL\Classif_Bi_Dir_GRU_Model\Trained_models\Baseline\Period_1\1st_try\BiGRUWithAttention_epoch_1987.pth


### Period 3 --> Training and saving in __*'1st_try'*__ (BiGRUWithLoRA, lora_r=4) ---> Val acc = 90.83 %
### Val-Class-Acc: {0: '44.23%', 1: '96.89%', 2: '91.55%', 3: '95.27%'}
```
Class Distribution for 'y_val':

Class 0   Percent:  10.26% ||
Class 1   Percent:  46.19% ||
Class 2   Percent:   4.21% ||
Class 3   Percent:  39.34% 
```

In [121]:
"""
- 'trend': Categorized trend values based on the detected phases:
    - 0: No trend
    - 1: Moderate negative trend
    - 2: Very strong negative trend
    - 3: Moderate positive trend
    - 4: Very strong positive trend
"""
with contextlib.redirect_stdout(open(os.devnull, 'w')):
    X_train, y_train, X_val, y_val, X_test, y_test, Number_features = process_and_return_splits(
        with_indicators_file_path = list_period_files_full_path[2], # Change 
        downsampled_data_minutes = downsampled_data_minutes,
        exclude_columns = exclude_columns,
        lower_threshold = lower_threshold,
        upper_threshold = upper_threshold,
        reverse_steps = reverse_steps,
        sequence_length = sequence_length,
        sliding_interval = sliding_interval,
        trends_to_keep = {0, 1, 2, 3}  # Default keeps all trends : {0, 1, 2, 3, 4}
        # trends_to_keep = {0, 1, 2, 3, 4}  # Default keeps all trends
    )

print(f"\nNumber_features = {Number_features}")

unique_classes = np.unique(y_val)
num_classes = len(unique_classes)
print(f"unique_classes = {unique_classes}")
print(f"num_classes = {num_classes}")

print_class_distribution(y_train, "y_train")
print_class_distribution(y_val, "y_val")
print_class_distribution(y_test, "y_test")



Number_features = 7
unique_classes = [0 1 2 3]
num_classes = 4
Class Distribution for 'y_train':       Class 0   Percent:  13.79% || Class 1   Percent:  37.59% || Class 2   Percent:   8.89% || Class 3   Percent:  39.73%
Class Distribution for 'y_val':         Class 0   Percent:  10.26% || Class 1   Percent:  46.19% || Class 2   Percent:   4.21% || Class 3   Percent:  39.34%
Class Distribution for 'y_test':        Class 0   Percent:  10.30% || Class 1   Percent:  43.50% || Class 2   Percent:   8.50% || Class 3   Percent:  37.70%


In [122]:
best_overall = max(track_across_runs, key=lambda res: res['val_accuracy'])
best_model_path = best_overall['model_path']
best_model_path

'Class_Incremental_CL\\Classif_Bi_Dir_GRU_Model\\Trained_models\\LoRA\\Period_2\\1st_try\\BiGRUWithLoRA_epoch_629.pth'

In [123]:
# Model parameters
input_size = Number_features  # Number of features
hidden_size = 64  # Number of GRU units
output_size = num_classes # Must be dynamic, up to 5  # Number of trend classes (0, 15, 25, -15, -25)
num_layers = 4  # Number of GRU layers
dropout = 0.0
learning_rate = 0.0001
weight_decay=1e-5 # For the optimizer
use_scheduler = False
lora_r = 4 # Rank of the low-rank update matrices
# ---- ---- ---- #
num_epochs= 2000 # Number of epochs/ go through entire data
batch_size= 64 # How many sequences passed at once to the model
model_name = 'BiGRUWithLoRA' # Name of the model to use for saving
best_results = [] # Initialize this outside the training function or at the beginning of training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Define a global stop signal
stop_signal_file = os.path.normpath(os.path.join('Class_Incremental_CL', 'Classif_Bi_Dir_GRU_Model/stop_training.txt'))  # Create this file to stop training
model_saving_folder = os.path.normpath(os.path.join('Class_Incremental_CL', "Classif_Bi_Dir_GRU_Model/Trained_models/LoRA/Period_3/1st_try"))
ensure_folder(model_saving_folder)

#--------------------------------
# Instantiate the previous model
#--------------------------------
# (Re-)instantiate the base model (from period 1) with original settings.
base_model = BiGRUWithAttention(input_size=input_size, hidden_size=hidden_size, 
                                output_size=2,  # For period 1, originally 2 classes.
                                num_layers=num_layers, dropout=dropout)
# Instantiate the period 2 model.
previous_model = BiGRUWithLoRA(base_model, old_num_classes=2, new_total_classes=3, lora_rank=lora_r)
#-------------------------------------------------------------------------
best_overall = max(track_across_runs, key=lambda res: res['val_accuracy'])
best_model_path = best_overall['model_path']
#----------------------------------------------------------------------
# Initialize the list to store results across runs
track_across_runs = []
#----------------------------------------------------------------------
#-------------------------------------------------------------------------
previous_model_checkpoint_path = os.path.normpath(best_model_path)
previous_model_checkpoint = torch.load(previous_model_checkpoint_path, map_location=torch.device('cpu'), weights_only=True)
previous_model.load_state_dict(previous_model_checkpoint['model_state_dict'])
print(f"Loaded previous model from: \n\t{previous_model_checkpoint_path}")
print(f"\n{previous_model}\n")
del previous_model_checkpoint, previous_model_checkpoint_path
gc.collect()

#-------------------------------------------------------------------------
# Instantiate the current model
current_model = BiGRUWithLoRA(previous_model, old_num_classes=output_size-1, new_total_classes=output_size, lora_rank=lora_r).to(device)
#-------------------------------------------------------------------------

# Define the loss function, optimizer and scheduler
criterion = nn.CrossEntropyLoss()
# Only parameters with requires_grad=True (the LoRA adapters) will be updated.
# Only parameters of the new head (fc_expanded, lora_A, lora_B) are trainable.
optimizer = optim.Adam(filter(lambda p: p.requires_grad, 
                              current_model.parameters()), 
                              lr=learning_rate, 
                              weight_decay=weight_decay)
# optimizer = optim.Adam(class_gru_model.parameters(), lr=0.001, weight_decay=1e-5)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

train_and_validate(
    model=current_model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val,
    scheduler=scheduler,
    use_scheduler=use_scheduler,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name, # Use base name for saved files in this period
    stop_signal_file=stop_signal_file
)

#----------------------------------------------------------------------
# Append only the best result (already at index 0)
track_across_runs.append(best_results[0])
#----------------------------------------------------------------------

for res in best_results:        
    print(f"Epoch {res['epoch']}/{num_epochs}, "
            f"Train Loss: {res['train_loss']:.4f}, " 
            f"Val Loss: {res['val_loss']:.4f}, "
            f"Val Accuracy: {res['val_accuracy'] * 100:.2f}%, "
            f"Model Path: {res['model_path']}")      
print(f"\ncurrent_model: \n{current_model}")

print(f"\nunique_classes = {unique_classes}")
print(f"num_classes = {num_classes}")

for var in [
    "X_train", "y_train", "X_val", "y_val", "X_test", "y_test",
    "Number_features", "unique_classes", "num_classes",
    "base_model", "previous_model", "current_model",
    "criterion", "optimizer", "scheduler", 
]:
    if var in locals():
        del locals()[var]
# --- Force garbage collection ---
torch.cuda.empty_cache()
gc.collect()


Loaded previous model from: 
	Class_Incremental_CL\Classif_Bi_Dir_GRU_Model\Trained_models\LoRA\Period_2\1st_try\BiGRUWithLoRA_epoch_629.pth

BiGRUWithLoRA(
  (base_model): BiGRUWithAttention(
    (gru): GRU(7, 64, num_layers=4, batch_first=True, bidirectional=True)
    (attention_fc): Linear(in_features=128, out_features=128, bias=True)
    (fc): Linear(in_features=128, out_features=2, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (fc_expanded): Linear(in_features=128, out_features=3, bias=True)
)

'train_and_validate' function started. 

Existing folder has been removed : Class_Incremental_CL\Classif_Bi_Dir_GRU_Model\Trained_models\LoRA\Period_3\1st_try

y_train:
<class 'torch.Tensor'>
torch.int64
torch.Size([3634, 1000])
X_train:
<class 'torch.Tensor'>
torch.float32
torch.Size([3634, 1000, 7])

y_val:
<class 'torch.Tensor'>
torch.int64
torch.Size([454, 1000])
X_val:
<class 'torch.Tensor'>
torch.float32
torch.Size([454, 1000, 7])

Dataset Lengths:
Train Dataset Length

0

In [124]:
print(best_model_path)

Class_Incremental_CL\Classif_Bi_Dir_GRU_Model\Trained_models\LoRA\Period_2\1st_try\BiGRUWithLoRA_epoch_629.pth


### Period 4 --> Training and saving in __*'1st_try'*__ (BiGRUWithLoRA, lora_r=4) ---> Val acc = 86.29 %
### Val-Class-Acc: {0: '63.77%', 1: '94.94%', 2: '86.68%', 3: '93.96%', 4: '38.32%'}
```
Class Distribution for 'y_val':

Class 0   Percent:   5.35% ||
Class 1   Percent:  36.15% ||
Class 2   Percent:  14.78% ||
Class 3   Percent:  34.02% ||
Class 4   Percent:   9.70% 
```

In [125]:
"""
- 'trend': Categorized trend values based on the detected phases:
    - 0: No trend
    - 1: Moderate negative trend
    - 2: Very strong negative trend
    - 3: Moderate positive trend
    - 4: Very strong positive trend
"""
with contextlib.redirect_stdout(open(os.devnull, 'w')):
    X_train, y_train, X_val, y_val, X_test, y_test, Number_features = process_and_return_splits(
        with_indicators_file_path = list_period_files_full_path[3], # Change 
        downsampled_data_minutes = downsampled_data_minutes,
        exclude_columns = exclude_columns,
        lower_threshold = lower_threshold,
        upper_threshold = upper_threshold,
        reverse_steps = reverse_steps,
        sequence_length = sequence_length,
        sliding_interval = sliding_interval,
        trends_to_keep = {0, 1, 2, 3, 4}  # Default keeps all trends : {0, 1, 2, 3, 4}
        # trends_to_keep = {0, 1, 2, 3, 4}  # Default keeps all trends
    )

print(f"\nNumber_features = {Number_features}")

unique_classes = np.unique(y_val)
num_classes = len(unique_classes)
print(f"unique_classes = {unique_classes}")
print(f"num_classes = {num_classes}")

print_class_distribution(y_train, "y_train")
print_class_distribution(y_val, "y_val")
print_class_distribution(y_test, "y_test")



Number_features = 7
unique_classes = [0 1 2 3 4]
num_classes = 5
Class Distribution for 'y_train':       Class 0   Percent:   6.47% || Class 1   Percent:  39.17% || Class 2   Percent:   7.24% || Class 3   Percent:  41.06% || Class 4   Percent:   6.06%
Class Distribution for 'y_val':         Class 0   Percent:   5.35% || Class 1   Percent:  36.15% || Class 2   Percent:  14.78% || Class 3   Percent:  34.02% || Class 4   Percent:   9.70%
Class Distribution for 'y_test':        Class 0   Percent:   6.63% || Class 1   Percent:  40.53% || Class 2   Percent:   5.48% || Class 3   Percent:  41.29% || Class 4   Percent:   6.08%


In [126]:
best_overall = max(track_across_runs, key=lambda res: res['val_accuracy'])
best_model_path = best_overall['model_path']
best_model_path

'Class_Incremental_CL\\Classif_Bi_Dir_GRU_Model\\Trained_models\\LoRA\\Period_3\\1st_try\\BiGRUWithLoRA_epoch_95.pth'

In [127]:
# Model parameters
input_size = Number_features  # Number of features
hidden_size = 64  # Number of GRU units
output_size = num_classes # Must be dynamic, up to 5  # Number of trend classes (0, 15, 25, -15, -25)
num_layers = 4  # Number of GRU layers
dropout = 0.0
learning_rate = 0.0001
weight_decay=1e-5 # For the optimizer
use_scheduler = False
lora_r = 4 # Rank of the low-rank update matrices
# ---- ---- ---- #
num_epochs= 2000 # Number of epochs/ go through entire data
batch_size= 64 # How many sequences passed at once to the model
model_name = 'BiGRUWithLoRA' # Name of the model to use for saving
best_results = [] # Initialize this outside the training function or at the beginning of training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Define a global stop signal
stop_signal_file = os.path.normpath(os.path.join('Class_Incremental_CL', 'Classif_Bi_Dir_GRU_Model/stop_training.txt'))  # Create this file to stop training
model_saving_folder = os.path.normpath(os.path.join('Class_Incremental_CL', "Classif_Bi_Dir_GRU_Model/Trained_models/LoRA/Period_4/1st_try"))
ensure_folder(model_saving_folder)

#--------------------------------
# Instantiate the previous model
#--------------------------------
# (Re-)instantiate the base model (from period 1) with original settings.
base_model = BiGRUWithAttention(input_size=input_size, hidden_size=hidden_size, 
                                output_size=2,  # For period 1, originally 2 classes.
                                num_layers=num_layers, dropout=dropout)
# Instantiate the period 2 model.
period_2_model = BiGRUWithLoRA(base_model, old_num_classes=2, new_total_classes=3, lora_rank=lora_r)
# Instantiate the period 3 model.
previous_model = BiGRUWithLoRA(period_2_model, old_num_classes=3, new_total_classes=4, lora_rank=lora_r)
#-------------------------------------------------------------------------
best_overall = max(track_across_runs, key=lambda res: res['val_accuracy'])
best_model_path = best_overall['model_path']
#----------------------------------------------------------------------
# Initialize the list to store results across runs
track_across_runs = []
#----------------------------------------------------------------------
#-------------------------------------------------------------------------
previous_model_checkpoint_path = os.path.normpath(best_model_path)
previous_model_checkpoint = torch.load(previous_model_checkpoint_path, map_location=torch.device('cpu'), weights_only=True)
previous_model.load_state_dict(previous_model_checkpoint['model_state_dict'])
print(f"Loaded previous model from: \n\t{previous_model_checkpoint_path}")
print(f"\n{previous_model}\n")
del previous_model_checkpoint, previous_model_checkpoint_path
gc.collect()

#-------------------------------------------------------------------------
# Instantiate the current model
current_model = BiGRUWithLoRA(previous_model, old_num_classes=output_size-1, new_total_classes=output_size, lora_rank=lora_r).to(device)
#-------------------------------------------------------------------------

# Define the loss function, optimizer and scheduler
criterion = nn.CrossEntropyLoss()
# Only parameters with requires_grad=True (the LoRA adapters) will be updated.
# Only parameters of the new head (fc_expanded, lora_A, lora_B) are trainable.
optimizer = optim.Adam(filter(lambda p: p.requires_grad, 
                              current_model.parameters()), 
                              lr=learning_rate, 
                              weight_decay=weight_decay)
# optimizer = optim.Adam(class_gru_model.parameters(), lr=0.001, weight_decay=1e-5)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

train_and_validate(
    model=current_model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val,
    scheduler=scheduler,
    use_scheduler=use_scheduler,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name=model_name, # Use base name for saved files in this period
    stop_signal_file=stop_signal_file
)

#----------------------------------------------------------------------
# Append only the best result (already at index 0)
track_across_runs.append(best_results[0])
#----------------------------------------------------------------------

for res in best_results:        
    print(f"Epoch {res['epoch']}/{num_epochs}, "
            f"Train Loss: {res['train_loss']:.4f}, " 
            f"Val Loss: {res['val_loss']:.4f}, "
            f"Val Accuracy: {res['val_accuracy'] * 100:.2f}%, "
            f"Model Path: {res['model_path']}")      
print(f"\ncurrent_model: \n{current_model}")

print(f"\nunique_classes = {unique_classes}")
print(f"num_classes = {num_classes}")

for var in [
    "X_train", "y_train", "X_val", "y_val", "X_test", "y_test",
    "Number_features", "unique_classes", "num_classes",
    "base_model", "previous_model", "current_model",
    "criterion", "optimizer", "scheduler", 
]:
    if var in locals():
        del locals()[var]
# --- Force garbage collection ---
torch.cuda.empty_cache()
gc.collect()


Loaded previous model from: 
	Class_Incremental_CL\Classif_Bi_Dir_GRU_Model\Trained_models\LoRA\Period_3\1st_try\BiGRUWithLoRA_epoch_95.pth

BiGRUWithLoRA(
  (base_model): BiGRUWithAttention(
    (gru): GRU(7, 64, num_layers=4, batch_first=True, bidirectional=True)
    (attention_fc): Linear(in_features=128, out_features=128, bias=True)
    (fc): Linear(in_features=128, out_features=2, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (fc_expanded): Linear(in_features=128, out_features=4, bias=True)
)

'train_and_validate' function started. 

Existing folder has been removed : Class_Incremental_CL\Classif_Bi_Dir_GRU_Model\Trained_models\LoRA\Period_4\1st_try

y_train:
<class 'torch.Tensor'>
torch.int64
torch.Size([3634, 1000])
X_train:
<class 'torch.Tensor'>
torch.float32
torch.Size([3634, 1000, 7])

y_val:
<class 'torch.Tensor'>
torch.int64
torch.Size([454, 1000])
X_val:
<class 'torch.Tensor'>
torch.float32
torch.Size([454, 1000, 7])

Dataset Lengths:
Train Dataset Length:

0

In [128]:
print(best_model_path)

Class_Incremental_CL\Classif_Bi_Dir_GRU_Model\Trained_models\LoRA\Period_3\1st_try\BiGRUWithLoRA_epoch_95.pth
